In [24]:
#Importar librerías iniciales
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [25]:
#Importar dataframe desde Excel
path_reporte_bruto = r"C:\Users\bgacitua\Desktop\Automatización HHEE\Reportes bono asistencia\2025-08-11 08_45_27-1197135513-reporte_turnos_bruto_combinado.xlsx"
df = pd.read_excel(path_reporte_bruto)

display((df.head()))



,idturno,nombreTipoAusencia,nombreUnidad,Centrocosto_turno,nombreSector,nombreDetPuesto,nombreTipoRotativa,nombreTipoTurno,nombreDetTurno,fecha,...,apellidoPaternoCubre,apellidoMaternoCubre,rutAusente,nombreAusente,apellidoPaternoAusente,apellidoMaternoAusente,adminCreador,adminModificador,fechaCreacion,fechaModificacion
0,478781,,Administación y Lavandería,Administración,HSolar,Secretaria,Lu@ju 07:50@17 + Vi 07:50@15:30,T 7:50@17,normal,2025-07-07,...,,,,,,,,,,
1,478782,,Administación y Lavandería,Administración,HSolar,Secretaria,Lu@ju 07:50@17 + Vi 07:50@15:30,T 7:50@17,normal,2025-07-08,...,,,,,,,,,,
2,478783,,Administación y Lavandería,Administración,HSolar,Secretaria,Lu@ju 07:50@17 + Vi 07:50@15:30,T 7:50@17,normal,2025-07-09,...,,,,,,,,,,
3,478784,,Administación y Lavandería,Administración,HSolar,Secretaria,Lu@ju 07:50@17 + Vi 07:50@15:30,T 7:50@17,normal,2025-07-10,...,,,,,,,,,,
4,478785,,Administación y Lavandería,Administración,HSolar,Secretaria,Lu@ju 07:50@17 + Vi 07:50@15:30,D 7:50@15:30,normal,2025-07-11,...,,,,,,,,,,


In [26]:
# Filtrar columnas específicas
columnas_deseadas = [
    'fecha', 
    'rut', 
    'nombre', 
    'apellidoPaterno', 
    'apellidoMaterno', 
    'PuestoIntegracion_trabajador', 
    'observacion', 
    'horaInicio', 
    'horaProgramadaInicio', 
    'HoraInicioMarcaReloj', 
    'horaTermino', 
    'horaProgramadaTermino', 
    'HoraTerminoMarcaReloj', 
    'Centrocosto_trabajador', 
    'nombreMotivoPago',
    'nombreTipoAusencia'
]

df_filtrado = df[columnas_deseadas]
print(f"Forma del DataFrame filtrado: {df_filtrado.shape}")
display(df_filtrado.head(2))

Forma del DataFrame filtrado: (10647, 16)


,fecha,rut,nombre,apellidoPaterno,apellidoMaterno,PuestoIntegracion_trabajador,observacion,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,Centrocosto_trabajador,nombreMotivoPago,nombreTipoAusencia
0,2025-07-07,69866409,Gloria,Gonzalez,Castro,Secretaria,,07:50:00,2025-07-07 07:50:00,2025-07-07 07:31:39,17:00:00,2025-07-07 17:00:00,2025-07-07 16:55:15,Administración,,
1,2025-07-08,69866409,Gloria,Gonzalez,Castro,Secretaria,,07:50:00,2025-07-08 07:50:00,2025-07-08 06:57:57,17:00:00,2025-07-08 17:00:00,2025-07-08 16:56:27,Administración,,


In [27]:
# Filtrar por puestos específicos que están sujetos a bono antes de contar licencias
puestos_deseados = [
    'Analista De Control De Calidad', 
    'Analista De Microbiología', 
    'Asistente De Laboratorio', 
    'Coordinadora Muestras', 
    'Inspector De Proceso',
    'Administrativo Bodega Despacho', 
    'Ayudante De Bodega', 
    'Asistente De Bodega', 
    'Chofer Administrativo Transporte', 
    'Chofer', 
    'Coordinador de Planta', 
    'Encargado De Bodega Inflamables', 
    'Encargado De Bodega Materias Primas', 
    'Operario', 
    'Operario Almacenamiento y Gestión de Residuos', 
    'Peoneta',
    'Asistente De Servicios Generales'
]

# RUT específico a excluir del puesto "Asistente De Servicios Generales" - Cesar Vera
rut_excluido = '128294988'

# Filtrar DataFrame por puestos específicos
df_filtrado_puestos = df_filtrado[df_filtrado['PuestoIntegracion_trabajador'].isin(puestos_deseados)]

# Excluir el RUT específico del puesto "Asistente De Servicios Generales"
df_filtrado_puestos = df_filtrado_puestos[
    ~((df_filtrado_puestos['PuestoIntegracion_trabajador'] == 'Asistente De Servicios Generales') & 
    (df_filtrado_puestos['rut'] == rut_excluido))
]
print(f"Registros después de filtrar por puesto: {df_filtrado_puestos.shape}") #DF con puestos específicos y sin César

# Crear columna Licencia_1ra_qcna
# Definir los tipos de licencia a contar
tipos_licencia = ["Licencia", "Licencia Accidente Trayecto", "Licencia Accidente Trabajo", "Licencia Niño Menor", "Licencia postnatal"]

# Obtener el rango de fechas del archivo filtrado
fecha_min = df_filtrado_puestos['fecha'].min()
fecha_max = df_filtrado_puestos['fecha'].max()
print(f"Rango de fechas: {fecha_min} a {fecha_max}")

# Contar licencias por rut en el rango de fechas (solo para puestos filtrados)
licencias_por_rut = df_filtrado_puestos[
    df_filtrado_puestos['nombreTipoAusencia'].isin(tipos_licencia)
].groupby('rut').size().reset_index(name='conteo_licencias')

# Hacer merge con el dataframe filtrado por puestos
df_filtrado_puestos = df_filtrado_puestos.merge(licencias_por_rut, on='rut', how='left')

# Renombrar la columna y llenar valores nulos con 0
df_filtrado_puestos['Licencia_1ra_qcna'] = df_filtrado_puestos['conteo_licencias'].fillna(0).astype(int)
df_filtrado_puestos.drop('conteo_licencias', axis=1, inplace=True)

print(f"Forma del DataFrame final: {df_filtrado_puestos.shape}")
print(f"Puestos considerados: {len(puestos_deseados)} puestos")
print(f"Tipos de licencia considerados: {tipos_licencia}")
display(df_filtrado_puestos[['rut', 'nombre', 'PuestoIntegracion_trabajador', 'nombreTipoAusencia', 'Licencia_1ra_qcna']])

Registros después de filtrar por puesto: (5835, 16)
Rango de fechas: 2025-07-06 a 2025-08-05
Forma del DataFrame final: (5835, 17)
Puestos considerados: 17 puestos
Tipos de licencia considerados: ['Licencia', 'Licencia Accidente Trayecto', 'Licencia Accidente Trabajo', 'Licencia Niño Menor', 'Licencia postnatal']


,rut,nombre,PuestoIntegracion_trabajador,nombreTipoAusencia,Licencia_1ra_qcna
0,128294988,Cesar,Asistente De Servicios Generales,,0
1,128294988,Cesar,Asistente De Servicios Generales,,0
2,128294988,Cesar,Asistente De Servicios Generales,,0
3,128294988,Cesar,Asistente De Servicios Generales,,0
4,128294988,Cesar,Asistente De Servicios Generales,,0
...,...,...,...,...,...
5830,212892254,Paz,Asistente De Laboratorio,,0
5831,212892254,Paz,Asistente De Laboratorio,,0
5832,212892254,Paz,Asistente De Laboratorio,,0
5833,212892254,Paz,Asistente De Laboratorio,,0


In [28]:
# Crear columnas de licencias por periodos
# Definir los periodos
periodo_1_inicio = pd.to_datetime('2025-07-06')
periodo_1_fin = pd.to_datetime('2025-07-20')
periodo_2_inicio = pd.to_datetime('2025-07-21')
periodo_2_fin = pd.to_datetime('2025-08-05')

print(f"Periodo 1: {periodo_1_inicio.strftime('%d-%m-%Y')} al {periodo_1_fin.strftime('%d-%m-%Y')}")
print(f"Periodo 2: {periodo_2_inicio.strftime('%d-%m-%Y')} al {periodo_2_fin.strftime('%d-%m-%Y')}")

# Convertir la columna fecha a datetime si no lo está
df_filtrado_puestos['fecha'] = pd.to_datetime(df_filtrado_puestos['fecha'])

# Conteo de licencias para el periodo 1 (06-07-2025 al 20-07-2025)
licencias_periodo_1 = df_filtrado_puestos[
    (df_filtrado_puestos['fecha'] >= periodo_1_inicio) & 
    (df_filtrado_puestos['fecha'] <= periodo_1_fin) &
    (df_filtrado_puestos['nombreTipoAusencia'].isin(tipos_licencia))
].groupby('rut').size().reset_index(name='Licencias_periodo_1')

# Conteo de licencias para el periodo 2 (21-07-2025 al 05-08-2025)
licencias_periodo_2 = df_filtrado_puestos[
    (df_filtrado_puestos['fecha'] >= periodo_2_inicio) & 
    (df_filtrado_puestos['fecha'] <= periodo_2_fin) &
    (df_filtrado_puestos['nombreTipoAusencia'].isin(tipos_licencia))
].groupby('rut').size().reset_index(name='Licencias_periodo_2')

# Obtener lista única de RUTs en el dataframe filtrado
ruts_unicos = df_filtrado_puestos['rut'].unique()
df_ruts = pd.DataFrame({'rut': ruts_unicos})

# Hacer merge con los conteos por periodo
df_ruts = df_ruts.merge(licencias_periodo_1, on='rut', how='left')
df_ruts = df_ruts.merge(licencias_periodo_2, on='rut', how='left')

# Rellenar valores nulos con 0
df_ruts['Licencias_periodo_1'] = df_ruts['Licencias_periodo_1'].fillna(0).astype(int)
df_ruts['Licencias_periodo_2'] = df_ruts['Licencias_periodo_2'].fillna(0).astype(int)

# Hacer merge con el dataframe principal
df_filtrado_puestos = df_filtrado_puestos.merge(df_ruts, on='rut', how='left')

print(f"\nResumen de licencias por periodo:")
print(f"Trabajadores con licencias en periodo 1: {(df_ruts['Licencias_periodo_1'] > 0).sum()}")
print(f"Trabajadores con licencias en periodo 2: {(df_ruts['Licencias_periodo_2'] > 0).sum()}")
print(f"Total de licencias periodo 1: {df_ruts['Licencias_periodo_1'].sum()}")
print(f"Total de licencias periodo 2: {df_ruts['Licencias_periodo_2'].sum()}")

# Mostrar muestra de los datos con las nuevas columnas
display(df_filtrado_puestos[['rut', 'nombre', 'PuestoIntegracion_trabajador', 'Licencia_1ra_qcna', 'Licencias_periodo_1', 'Licencias_periodo_2']].drop_duplicates('rut').head(10))

Periodo 1: 06-07-2025 al 20-07-2025
Periodo 2: 21-07-2025 al 05-08-2025

Resumen de licencias por periodo:
Trabajadores con licencias en periodo 1: 11
Trabajadores con licencias en periodo 2: 11
Total de licencias periodo 1: 78
Total de licencias periodo 2: 111


,rut,nombre,PuestoIntegracion_trabajador,Licencia_1ra_qcna,Licencias_periodo_1,Licencias_periodo_2
0,128294988,Cesar,Asistente De Servicios Generales,0,0,0
21,256248786,Marie,Asistente De Servicios Generales,0,0,0
26,107806466,Rosa,Asistente De Servicios Generales,0,0,0
31,82755640,Marcela,Asistente De Servicios Generales,0,0,0
37,116207923,Rosa,Asistente De Servicios Generales,0,0,0
108,261696436,Eyver,Operario,0,0,0
129,158229838,Carlos,Operario,0,0,0
152,268128743,Renato,Inspector De Proceso,0,0,0
173,158929724,Maritza,Inspector De Proceso,21,9,12
194,270363067,Jesus,Inspector De Proceso,21,9,12


In [ ]:
# Definir los tipos de permisos a contar
tipos_permisos = ["Permiso con goce de sueldo", "Permiso por Fallecimiento"]

In [30]:
# Crear columnas de permisos por periodos
# Usar los mismos periodos definidos anteriormente
periodo_1_inicio = pd.to_datetime('2025-07-06')
periodo_1_fin = pd.to_datetime('2025-07-20')
periodo_2_inicio = pd.to_datetime('2025-07-21')
periodo_2_fin = pd.to_datetime('2025-08-05')

print(f"Contando permisos para:")
print(f"Periodo 1: {periodo_1_inicio.strftime('%d-%m-%Y')} al {periodo_1_fin.strftime('%d-%m-%Y')}")
print(f"Periodo 2: {periodo_2_inicio.strftime('%d-%m-%Y')} al {periodo_2_fin.strftime('%d-%m-%Y')}")
print(f"Tipos de permisos: {tipos_permisos}")

# Conteo de permisos para el periodo 1 (06-07-2025 al 20-07-2025)
permisos_periodo_1 = df_filtrado_puestos[
    (df_filtrado_puestos['fecha'] >= periodo_1_inicio) & 
    (df_filtrado_puestos['fecha'] <= periodo_1_fin) &
    (df_filtrado_puestos['nombreTipoAusencia'].isin(tipos_permisos))
].groupby('rut').size().reset_index(name='Permisos_periodo_1')

# Conteo de permisos para el periodo 2 (21-07-2025 al 05-08-2025)
permisos_periodo_2 = df_filtrado_puestos[
    (df_filtrado_puestos['fecha'] >= periodo_2_inicio) & 
    (df_filtrado_puestos['fecha'] <= periodo_2_fin) &
    (df_filtrado_puestos['nombreTipoAusencia'].isin(tipos_permisos))
].groupby('rut').size().reset_index(name='Permisos_periodo_2')

# Obtener lista única de RUTs en el dataframe filtrado (si no existe ya)
if 'df_ruts_permisos' not in locals():
    ruts_unicos = df_filtrado_puestos['rut'].unique()
    df_ruts_permisos = pd.DataFrame({'rut': ruts_unicos})
else:
    df_ruts_permisos = pd.DataFrame({'rut': df_filtrado_puestos['rut'].unique()})

# Hacer merge con los conteos por periodo
df_ruts_permisos = df_ruts_permisos.merge(permisos_periodo_1, on='rut', how='left')
df_ruts_permisos = df_ruts_permisos.merge(permisos_periodo_2, on='rut', how='left')

# Rellenar valores nulos con 0
df_ruts_permisos['Permisos_periodo_1'] = df_ruts_permisos['Permisos_periodo_1'].fillna(0).astype(int)
df_ruts_permisos['Permisos_periodo_2'] = df_ruts_permisos['Permisos_periodo_2'].fillna(0).astype(int)

# Hacer merge con el dataframe principal
df_filtrado_puestos = df_filtrado_puestos.merge(df_ruts_permisos[['rut', 'Permisos_periodo_1', 'Permisos_periodo_2']], on='rut', how='left')

print(f"\nResumen de permisos por periodo:")
print(f"Trabajadores con permisos en periodo 1: {(df_ruts_permisos['Permisos_periodo_1'] > 0).sum()}")
print(f"Trabajadores con permisos en periodo 2: {(df_ruts_permisos['Permisos_periodo_2'] > 0).sum()}")
print(f"Total de permisos periodo 1: {df_ruts_permisos['Permisos_periodo_1'].sum()}")
print(f"Total de permisos periodo 2: {df_ruts_permisos['Permisos_periodo_2'].sum()}")

# Mostrar muestra de los datos con las nuevas columnas de permisos
display(df_filtrado_puestos[['rut', 'nombre', 'PuestoIntegracion_trabajador', 'Permisos_periodo_1', 'Permisos_periodo_2']].drop_duplicates('rut').head(10))

Contando permisos para:
Periodo 1: 06-07-2025 al 20-07-2025
Periodo 2: 21-07-2025 al 05-08-2025
Tipos de permisos: ['Permiso con goce de sueldo', 'Permiso por Fallecimiento', 'Permiso por matrimonio']

Resumen de permisos por periodo:
Trabajadores con permisos en periodo 1: 3
Trabajadores con permisos en periodo 2: 5
Total de permisos periodo 1: 3
Total de permisos periodo 2: 10


,rut,nombre,PuestoIntegracion_trabajador,Permisos_periodo_1,Permisos_periodo_2
0,128294988,Cesar,Asistente De Servicios Generales,0,0
21,256248786,Marie,Asistente De Servicios Generales,0,0
26,107806466,Rosa,Asistente De Servicios Generales,0,0
31,82755640,Marcela,Asistente De Servicios Generales,0,0
37,116207923,Rosa,Asistente De Servicios Generales,0,0
108,261696436,Eyver,Operario,0,0
129,158229838,Carlos,Operario,0,0
152,268128743,Renato,Inspector De Proceso,0,0
173,158929724,Maritza,Inspector De Proceso,0,0
194,270363067,Jesus,Inspector De Proceso,0,0


In [31]:
ausencia_injustificada = ["Inasistencia"]

In [32]:
# Crear columnas de ausencias injustificadas por periodos
# Usar los mismos periodos definidos anteriormente
periodo_1_inicio = pd.to_datetime('2025-07-06')
periodo_1_fin = pd.to_datetime('2025-07-20')
periodo_2_inicio = pd.to_datetime('2025-07-21')
periodo_2_fin = pd.to_datetime('2025-08-05')

print(f"Contando ausencias injustificadas para:")
print(f"Periodo 1: {periodo_1_inicio.strftime('%d-%m-%Y')} al {periodo_1_fin.strftime('%d-%m-%Y')}")
print(f"Periodo 2: {periodo_2_inicio.strftime('%d-%m-%Y')} al {periodo_2_fin.strftime('%d-%m-%Y')}")
print(f"Tipos de ausencia injustificada: {ausencia_injustificada}")

# Conteo de ausencias injustificadas para el periodo 1 (06-07-2025 al 20-07-2025)
ausencias_periodo_1 = df_filtrado_puestos[
    (df_filtrado_puestos['fecha'] >= periodo_1_inicio) & 
    (df_filtrado_puestos['fecha'] <= periodo_1_fin) &
    (df_filtrado_puestos['nombreTipoAusencia'].isin(ausencia_injustificada))
].groupby('rut').size().reset_index(name='Ausencias_injust_periodo_1')

# Conteo de ausencias injustificadas para el periodo 2 (21-07-2025 al 05-08-2025)
ausencias_periodo_2 = df_filtrado_puestos[
    (df_filtrado_puestos['fecha'] >= periodo_2_inicio) & 
    (df_filtrado_puestos['fecha'] <= periodo_2_fin) &
    (df_filtrado_puestos['nombreTipoAusencia'].isin(ausencia_injustificada))
].groupby('rut').size().reset_index(name='Ausencias_injust_periodo_2')

# Obtener lista única de RUTs en el dataframe filtrado
ruts_unicos = df_filtrado_puestos['rut'].unique()
df_ruts_ausencias = pd.DataFrame({'rut': ruts_unicos})

# Hacer merge con los conteos por periodo
df_ruts_ausencias = df_ruts_ausencias.merge(ausencias_periodo_1, on='rut', how='left')
df_ruts_ausencias = df_ruts_ausencias.merge(ausencias_periodo_2, on='rut', how='left')

# Rellenar valores nulos con 0
df_ruts_ausencias['Ausencias_injust_periodo_1'] = df_ruts_ausencias['Ausencias_injust_periodo_1'].fillna(0).astype(int)
df_ruts_ausencias['Ausencias_injust_periodo_2'] = df_ruts_ausencias['Ausencias_injust_periodo_2'].fillna(0).astype(int)

# Hacer merge con el dataframe principal
df_filtrado_puestos = df_filtrado_puestos.merge(df_ruts_ausencias[['rut', 'Ausencias_injust_periodo_1', 'Ausencias_injust_periodo_2']], on='rut', how='left')

print(f"\nResumen de ausencias injustificadas por periodo:")
print(f"Trabajadores con ausencias injustificadas en periodo 1: {(df_ruts_ausencias['Ausencias_injust_periodo_1'] > 0).sum()}")
print(f"Trabajadores con ausencias injustificadas en periodo 2: {(df_ruts_ausencias['Ausencias_injust_periodo_2'] > 0).sum()}")
print(f"Total de ausencias injustificadas periodo 1: {df_ruts_ausencias['Ausencias_injust_periodo_1'].sum()}")
print(f"Total de ausencias injustificadas periodo 2: {df_ruts_ausencias['Ausencias_injust_periodo_2'].sum()}")

# Mostrar muestra de los datos con las nuevas columnas de ausencias injustificadas
display(df_filtrado_puestos[['rut', 'nombre', 'PuestoIntegracion_trabajador', 'Ausencias_injust_periodo_1', 'Ausencias_injust_periodo_2']].drop_duplicates('rut').head(10))

Contando ausencias injustificadas para:
Periodo 1: 06-07-2025 al 20-07-2025
Periodo 2: 21-07-2025 al 05-08-2025
Tipos de ausencia injustificada: ['Inasistencia']

Resumen de ausencias injustificadas por periodo:
Trabajadores con ausencias injustificadas en periodo 1: 0
Trabajadores con ausencias injustificadas en periodo 2: 6
Total de ausencias injustificadas periodo 1: 0
Total de ausencias injustificadas periodo 2: 6

Resumen de ausencias injustificadas por periodo:
Trabajadores con ausencias injustificadas en periodo 1: 0
Trabajadores con ausencias injustificadas en periodo 2: 6
Total de ausencias injustificadas periodo 1: 0
Total de ausencias injustificadas periodo 2: 6


,rut,nombre,PuestoIntegracion_trabajador,Ausencias_injust_periodo_1,Ausencias_injust_periodo_2
0,128294988,Cesar,Asistente De Servicios Generales,0,0
21,256248786,Marie,Asistente De Servicios Generales,0,0
26,107806466,Rosa,Asistente De Servicios Generales,0,0
31,82755640,Marcela,Asistente De Servicios Generales,0,0
37,116207923,Rosa,Asistente De Servicios Generales,0,0
108,261696436,Eyver,Operario,0,0
129,158229838,Carlos,Operario,0,0
152,268128743,Renato,Inspector De Proceso,0,0
173,158929724,Maritza,Inspector De Proceso,0,0
194,270363067,Jesus,Inspector De Proceso,0,0


In [33]:
# Automatización para detectar atrasos y omisiones de marca
from datetime import datetime, timedelta
import pandas as pd

print("=== DETECCIÓN DE ATRASOS Y OMISIONES DE MARCA ===")

# Crear una copia del dataframe para trabajar con atrasos
df_atrasos = df_filtrado_puestos.copy()

# Convertir las columnas de tiempo a datetime si no lo están
df_atrasos['fecha'] = pd.to_datetime(df_atrasos['fecha'])
df_atrasos['horaProgramadaInicio'] = pd.to_datetime(df_atrasos['horaProgramadaInicio'], errors='coerce')
df_atrasos['HoraInicioMarcaReloj'] = pd.to_datetime(df_atrasos['HoraInicioMarcaReloj'], errors='coerce')

# Función para detectar si es turno nocturno
def es_turno_nocturno(hora_programada):
    """
    Detecta si es un turno nocturno basado en la hora programada
    Considera turno nocturno si inicia entre 20:00 y 06:00
    """
    if pd.isna(hora_programada):
        return False
    hora = hora_programada.time()
    return hora.hour >= 20 or hora.hour <= 6

# Función para calcular el atraso considerando turnos nocturnos
def calcular_atraso(row):
    """
    Calcula el atraso en minutos considerando:
    1. Flexibilidad de 10 minutos
    2. Turnos nocturnos que pueden cruzar medianoche
    3. Omisiones de marca (solo si no hay justificación)
    4. Justificaciones en nombreTipoAusencia
    """
    fecha = row['fecha']
    hora_programada = row['horaProgramadaInicio']
    hora_marca = row['HoraInicioMarcaReloj']
    justificacion = row['nombreTipoAusencia']
    
    # Si no hay hora programada, no se puede calcular
    if pd.isna(hora_programada):
        return {'atraso_minutos': 0, 'tipo': 'Sin hora programada', 'es_nocturno': False}
    
    # Si no hay marca de reloj, verificar si hay justificación
    if pd.isna(hora_marca):
        # Si hay justificación (nombreTipoAusencia no está vacío), no es omisión
        if pd.notna(justificacion) and str(justificacion).strip() != '':
            return {'atraso_minutos': 0, 'tipo': 'Ausencia justificada', 'es_nocturno': es_turno_nocturno(hora_programada)}
        else:
            # Si no hay justificación, entonces es omisión de marca
            return {'atraso_minutos': 0, 'tipo': 'Omisión de marca', 'es_nocturno': es_turno_nocturno(hora_programada)}
    
    # Detectar si es turno nocturno
    turno_nocturno = es_turno_nocturno(hora_programada)
    
    # Crear datetime completo para hora programada del día actual
    hora_programada_completa = datetime.combine(fecha.date(), hora_programada.time())
    
    # Para turnos nocturnos, ajustar si es necesario
    if turno_nocturno:
        # Si la hora programada es muy temprana (00:00 - 06:00), podría ser del día siguiente
        if hora_programada.time().hour <= 6:
            # Mantener la fecha actual ya que el turno "comienza" este día
            pass
        else:
            # Hora programada es tarde (20:00-23:59), el turno comienza este día
            pass
    
    # Calcular límite de tolerancia (hora programada + 10 minutos)
    limite_tolerancia = hora_programada_completa + timedelta(minutes=10)
    
    # Para turnos nocturnos, considerar que la marca puede ser del día anterior
    if turno_nocturno and hora_marca.date() != fecha.date():
        # Si la marca es del día anterior y la hora programada es temprana del día siguiente
        if hora_programada.time().hour <= 6 and hora_marca.date() == (fecha.date() - timedelta(days=1)):
            # Ajustar la hora de marca al día actual para comparación
            hora_marca_ajustada = datetime.combine(fecha.date(), hora_marca.time())
            # Como cruzó medianoche, restar 24 horas a la hora programada para comparar
            hora_programada_ajustada = hora_programada_completa - timedelta(days=1)
            limite_tolerancia_ajustado = hora_programada_ajustada + timedelta(minutes=10)
            
            if hora_marca_ajustada <= limite_tolerancia_ajustado:
                return {'atraso_minutos': 0, 'tipo': 'Puntual', 'es_nocturno': True}
            else:
                atraso = (hora_marca_ajustada - limite_tolerancia_ajustado).total_seconds() / 60
                return {'atraso_minutos': max(0, atraso), 'tipo': 'Atraso', 'es_nocturno': True}
    
    # Caso normal: comparar horas del mismo día
    if hora_marca <= limite_tolerancia:
        return {'atraso_minutos': 0, 'tipo': 'Puntual', 'es_nocturno': turno_nocturno}
    else:
        atraso = (hora_marca - limite_tolerancia).total_seconds() / 60
        return {'atraso_minutos': atraso, 'tipo': 'Atraso', 'es_nocturno': turno_nocturno}

# Aplicar la función para calcular atrasos
print("Calculando atrasos y omisiones...")
resultados_atrasos = df_atrasos.apply(calcular_atraso, axis=1, result_type='expand')

# Agregar las columnas de resultado al dataframe
df_atrasos['atraso_minutos'] = resultados_atrasos['atraso_minutos'].round(2)
df_atrasos['tipo_asistencia'] = resultados_atrasos['tipo']
df_atrasos['es_turno_nocturno'] = resultados_atrasos['es_nocturno']

# Crear columna binaria para atrasos (1 si hay atraso, 0 si no)
df_atrasos['tiene_atraso'] = (df_atrasos['tipo_asistencia'] == 'Atraso').astype(int)

# Crear columna binaria para omisiones (1 si hay omisión, 0 si no)
df_atrasos['tiene_omision'] = (df_atrasos['tipo_asistencia'] == 'Omisión de marca').astype(int)

print("\n=== RESUMEN DE ATRASOS Y OMISIONES ===")
print(f"Total de registros analizados: {len(df_atrasos)}")
print(f"Registros puntuales: {(df_atrasos['tipo_asistencia'] == 'Puntual').sum()}")
print(f"Registros con atraso: {(df_atrasos['tipo_asistencia'] == 'Atraso').sum()}")
print(f"Registros con omisión de marca: {(df_atrasos['tipo_asistencia'] == 'Omisión de marca').sum()}")
print(f"Registros con ausencia justificada: {(df_atrasos['tipo_asistencia'] == 'Ausencia justificada').sum()}")
print(f"Registros sin hora programada: {(df_atrasos['tipo_asistencia'] == 'Sin hora programada').sum()}")
print(f"Turnos nocturnos detectados: {df_atrasos['es_turno_nocturno'].sum()}")

# Mostrar estadísticas de atrasos
atrasos_con_tiempo = df_atrasos[df_atrasos['atraso_minutos'] > 0]
if len(atrasos_con_tiempo) > 0:
    print(f"\nESTADÍSTICAS DE ATRASOS:")
    print(f"Atraso promedio: {atrasos_con_tiempo['atraso_minutos'].mean():.2f} minutos")
    print(f"Atraso máximo: {atrasos_con_tiempo['atraso_minutos'].max():.2f} minutos")
    print(f"Atraso mínimo: {atrasos_con_tiempo['atraso_minutos'].min():.2f} minutos")

# Mostrar distribución de tipos de ausencia justificada
ausencias_justificadas = df_atrasos[df_atrasos['tipo_asistencia'] == 'Ausencia justificada']
if len(ausencias_justificadas) > 0:
    print(f"\nTIPOS DE AUSENCIAS JUSTIFICADAS:")
    tipos_ausencia = ausencias_justificadas['nombreTipoAusencia'].value_counts()
    for tipo, cantidad in tipos_ausencia.items():
        print(f"  {tipo}: {cantidad} registros")

# Mostrar muestra de los resultados
print(f"\n=== MUESTRA DE RESULTADOS ===")
columnas_muestra = ['rut', 'nombre', 'fecha', 'horaProgramadaInicio', 'HoraInicioMarcaReloj', 
                'atraso_minutos', 'tipo_asistencia', 'es_turno_nocturno']
display(df_atrasos[columnas_muestra].head(10))

=== DETECCIÓN DE ATRASOS Y OMISIONES DE MARCA ===
Calculando atrasos y omisiones...
Calculando atrasos y omisiones...

=== RESUMEN DE ATRASOS Y OMISIONES ===
Total de registros analizados: 5835
Registros puntuales: 5223
Registros con atraso: 260
Registros con omisión de marca: 3
Registros con ausencia justificada: 349
Registros sin hora programada: 0
Turnos nocturnos detectados: 885

ESTADÍSTICAS DE ATRASOS:
Atraso promedio: 124.37 minutos
Atraso máximo: 2869.32 minutos
Atraso mínimo: 0.02 minutos

TIPOS DE AUSENCIAS JUSTIFICADAS:
  Licencia: 145 registros
  Vacación: 120 registros
  Licencia Accidente Trayecto: 21 registros
  Licencia postnatal: 21 registros
  Vacación Progresiva: 13 registros
  Permiso por Fallecimiento: 7 registros
  Licencia Accidente: 7 registros
  Inasistencia: 6 registros
  Permiso con goce de sueldo: 3 registros
  Permiso por matrimonio: 3 registros
  Licencia Accidente Trabajo: 2 registros
  201183677: 1 registros

=== MUESTRA DE RESULTADOS ===

=== RESUMEN DE

,rut,nombre,fecha,horaProgramadaInicio,HoraInicioMarcaReloj,atraso_minutos,tipo_asistencia,es_turno_nocturno
0,128294988,Cesar,2025-07-07,2025-07-07 07:50:00,2025-07-07 07:10:33,0.0,Puntual,False
1,128294988,Cesar,2025-07-08,2025-07-08 07:50:00,2025-07-08 07:18:48,0.0,Puntual,False
2,128294988,Cesar,2025-07-09,2025-07-09 07:50:00,2025-07-09 07:37:31,0.0,Puntual,False
3,128294988,Cesar,2025-07-10,2025-07-10 07:50:00,2025-07-10 07:23:20,0.0,Puntual,False
4,128294988,Cesar,2025-07-11,2025-07-11 07:50:00,2025-07-11 07:27:36,0.0,Puntual,False
5,128294988,Cesar,2025-07-14,2025-07-14 07:50:00,2025-07-14 07:12:04,0.0,Puntual,False
6,128294988,Cesar,2025-07-15,2025-07-15 07:50:00,2025-07-15 07:23:57,0.0,Puntual,False
7,128294988,Cesar,2025-07-17,2025-07-17 07:50:00,2025-07-17 07:46:23,0.0,Puntual,False
8,128294988,Cesar,2025-07-18,2025-07-18 07:50:00,2025-07-18 07:23:24,0.0,Puntual,False
9,128294988,Cesar,2025-07-21,2025-07-21 07:50:00,2025-07-21 07:19:14,0.0,Puntual,False


In [34]:
# Contar atrasos y omisiones por periodo
print("\n=== CONTEO DE ATRASOS Y OMISIONES POR PERIODO ===")

# Usar los mismos periodos definidos anteriormente
periodo_1_inicio = pd.to_datetime('2025-07-06')
periodo_1_fin = pd.to_datetime('2025-07-20')
periodo_2_inicio = pd.to_datetime('2025-07-21')
periodo_2_fin = pd.to_datetime('2025-08-05')

# Conteo de ATRASOS por periodo
# Periodo 1
atrasos_p1 = df_atrasos[
    (df_atrasos['fecha'] >= periodo_1_inicio) & 
    (df_atrasos['fecha'] <= periodo_1_fin) &
    (df_atrasos['tiene_atraso'] == 1)
].groupby('rut').size().reset_index(name='Atrasos_periodo_1')

# Periodo 2
atrasos_p2 = df_atrasos[
    (df_atrasos['fecha'] >= periodo_2_inicio) & 
    (df_atrasos['fecha'] <= periodo_2_fin) &
    (df_atrasos['tiene_atraso'] == 1)
].groupby('rut').size().reset_index(name='Atrasos_periodo_2')

# Conteo de OMISIONES por periodo
# Periodo 1
omisiones_p1 = df_atrasos[
    (df_atrasos['fecha'] >= periodo_1_inicio) & 
    (df_atrasos['fecha'] <= periodo_1_fin) &
    (df_atrasos['tiene_omision'] == 1)
].groupby('rut').size().reset_index(name='Omisiones_periodo_1')

# Periodo 2
omisiones_p2 = df_atrasos[
    (df_atrasos['fecha'] >= periodo_2_inicio) & 
    (df_atrasos['fecha'] <= periodo_2_fin) &
    (df_atrasos['tiene_omision'] == 1)
].groupby('rut').size().reset_index(name='Omisiones_periodo_2')

# Crear dataframe base con todos los RUTs
ruts_unicos = df_atrasos['rut'].unique()
df_ruts_asistencia = pd.DataFrame({'rut': ruts_unicos})

# Hacer merge con todos los conteos
df_ruts_asistencia = df_ruts_asistencia.merge(atrasos_p1, on='rut', how='left')
df_ruts_asistencia = df_ruts_asistencia.merge(atrasos_p2, on='rut', how='left')
df_ruts_asistencia = df_ruts_asistencia.merge(omisiones_p1, on='rut', how='left')
df_ruts_asistencia = df_ruts_asistencia.merge(omisiones_p2, on='rut', how='left')

# Rellenar valores nulos con 0
columnas_asistencia = ['Atrasos_periodo_1', 'Atrasos_periodo_2', 'Omisiones_periodo_1', 'Omisiones_periodo_2']
for col in columnas_asistencia:
    df_ruts_asistencia[col] = df_ruts_asistencia[col].fillna(0).astype(int)

# Hacer merge con el dataframe principal (df_filtrado_puestos)
df_filtrado_puestos = df_filtrado_puestos.merge(
    df_ruts_asistencia[['rut'] + columnas_asistencia], 
    on='rut', 
    how='left'
)

print(f"Periodo 1: {periodo_1_inicio.strftime('%d-%m-%Y')} al {periodo_1_fin.strftime('%d-%m-%Y')}")
print(f"Periodo 2: {periodo_2_inicio.strftime('%d-%m-%Y')} al {periodo_2_fin.strftime('%d-%m-%Y')}")

print(f"\nRESUMEN POR PERIODO:")
print(f"ATRASOS:")
print(f"  Trabajadores con atrasos periodo 1: {(df_ruts_asistencia['Atrasos_periodo_1'] > 0).sum()}")
print(f"  Trabajadores con atrasos periodo 2: {(df_ruts_asistencia['Atrasos_periodo_2'] > 0).sum()}")
print(f"  Total atrasos periodo 1: {df_ruts_asistencia['Atrasos_periodo_1'].sum()}")
print(f"  Total atrasos periodo 2: {df_ruts_asistencia['Atrasos_periodo_2'].sum()}")

print(f"\nOMISIONES:")
print(f"  Trabajadores con omisiones periodo 1: {(df_ruts_asistencia['Omisiones_periodo_1'] > 0).sum()}")
print(f"  Trabajadores con omisiones periodo 2: {(df_ruts_asistencia['Omisiones_periodo_2'] > 0).sum()}")
print(f"  Total omisiones periodo 1: {df_ruts_asistencia['Omisiones_periodo_1'].sum()}")
print(f"  Total omisiones periodo 2: {df_ruts_asistencia['Omisiones_periodo_2'].sum()}")

# Mostrar resumen final con todas las columnas nuevas
print(f"\n=== RESUMEN FINAL DEL DATAFRAME ===")
print(f"Columnas agregadas:")
print(f"- Licencias: Licencias_periodo_1, Licencias_periodo_2")
print(f"- Permisos: Permisos_periodo_1, Permisos_periodo_2") 
print(f"- Ausencias injustificadas: Ausencias_injust_periodo_1, Ausencias_injust_periodo_2")
print(f"- Atrasos: Atrasos_periodo_1, Atrasos_periodo_2")
print(f"- Omisiones: Omisiones_periodo_1, Omisiones_periodo_2")

# Mostrar muestra final
columnas_resumen = ['rut', 'nombre', 'PuestoIntegracion_trabajador', 
                   'Licencias_periodo_1', 'Licencias_periodo_2',
                   'Permisos_periodo_1', 'Permisos_periodo_2',
                   'Ausencias_injust_periodo_1', 'Ausencias_injust_periodo_2',
                   'Atrasos_periodo_1', 'Atrasos_periodo_2',
                   'Omisiones_periodo_1', 'Omisiones_periodo_2']

display(df_filtrado_puestos[columnas_resumen].drop_duplicates('rut').head(10))


=== CONTEO DE ATRASOS Y OMISIONES POR PERIODO ===
Periodo 1: 06-07-2025 al 20-07-2025
Periodo 2: 21-07-2025 al 05-08-2025

RESUMEN POR PERIODO:
ATRASOS:
  Trabajadores con atrasos periodo 1: 59
  Trabajadores con atrasos periodo 2: 66
  Total atrasos periodo 1: 106
  Total atrasos periodo 2: 154

OMISIONES:
  Trabajadores con omisiones periodo 1: 3
  Trabajadores con omisiones periodo 2: 0
  Total omisiones periodo 1: 3
  Total omisiones periodo 2: 0

=== RESUMEN FINAL DEL DATAFRAME ===
Columnas agregadas:
- Licencias: Licencias_periodo_1, Licencias_periodo_2
- Permisos: Permisos_periodo_1, Permisos_periodo_2
- Ausencias injustificadas: Ausencias_injust_periodo_1, Ausencias_injust_periodo_2
- Atrasos: Atrasos_periodo_1, Atrasos_periodo_2
- Omisiones: Omisiones_periodo_1, Omisiones_periodo_2


,rut,nombre,PuestoIntegracion_trabajador,Licencias_periodo_1,Licencias_periodo_2,Permisos_periodo_1,Permisos_periodo_2,Ausencias_injust_periodo_1,Ausencias_injust_periodo_2,Atrasos_periodo_1,Atrasos_periodo_2,Omisiones_periodo_1,Omisiones_periodo_2
0,128294988,Cesar,Asistente De Servicios Generales,0,0,0,0,0,0,0,0,0,0
21,256248786,Marie,Asistente De Servicios Generales,0,0,0,0,0,0,0,0,0,0
26,107806466,Rosa,Asistente De Servicios Generales,0,0,0,0,0,0,0,0,0,0
31,82755640,Marcela,Asistente De Servicios Generales,0,0,0,0,0,0,0,0,0,0
37,116207923,Rosa,Asistente De Servicios Generales,0,0,0,0,0,0,0,0,0,0
108,261696436,Eyver,Operario,0,0,0,0,0,0,0,0,0,0
129,158229838,Carlos,Operario,0,0,0,0,0,0,0,1,0,0
152,268128743,Renato,Inspector De Proceso,0,0,0,0,0,0,0,0,0,0
173,158929724,Maritza,Inspector De Proceso,9,12,0,0,0,0,0,0,0,0
194,270363067,Jesus,Inspector De Proceso,9,12,0,0,0,0,0,0,0,0


In [35]:
#Dataframe omisiones de marca

path_omisiones = r"C:\Users\bgacitua\Desktop\Automatización HHEE\Reportes bono asistencia\reporte_turnos_marcas_corregidas_20250811102841.xlsx"
df_reporte_omisiones = pd.read_excel(path_omisiones)
display(df_reporte_omisiones.head())

,Rut,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,27042560-7,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,15-07-2025,T,07:50,17:00,07:58:08,17:00:00 *,NaN,NaN,Olvido de Marcaje,No registra marcas en morpho,Solo salida
1,27042560-7,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,18-07-2025,D,07:50,14:30,08:00:23,14:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido de marca,Solo salida
2,27042560-7,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,05-08-2025,T,07:50,17:00,08:09:20,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido de marca,Solo salida
3,15797997-3,Débora Ester Muñoz Barrientos,Administración Gral De Producción,Ingeniero,07-07-2025,T,07:50,17:00,07:38:38,17:21:51,NaN,NaN,NaN,NaN,Entrada y salida
4,14194403-7,Manuel Jesus Abarca Moreira,Esencias y Env. Sabores,Operario,07-07-2025,T,07:50,17:00,07:50:00 *,16:55:31,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada


In [36]:
# Limpiar la columna Rut eliminando los guiones
print("=== LIMPIEZA DE COLUMNA RUT ===")
print(f"Forma del DataFrame antes de limpiar: {df_reporte_omisiones.shape}")

# Mostrar algunos ejemplos de RUTs antes de limpiar
print(f"Ejemplos de RUTs antes de limpiar:")
print(df_reporte_omisiones['Rut'].head(5).tolist())

# Eliminar guiones de la columna Rut
df_reporte_omisiones['Rut'] = df_reporte_omisiones['Rut'].astype(str).str.replace('-', '', regex=False)

# Mostrar algunos ejemplos de RUTs después de limpiar
print(f"\nEjemplos de RUTs después de limpiar:")
print(df_reporte_omisiones['Rut'].head(5).tolist())

print(f"Forma del DataFrame después de limpiar: {df_reporte_omisiones.shape}")

# Verificar que no queden guiones
ruts_con_guion = df_reporte_omisiones['Rut'].str.contains('-', na=False).sum()
print(f"RUTs que aún contienen guiones: {ruts_con_guion}")

# Mostrar muestra del dataframe limpio
display(df_reporte_omisiones.head())

=== LIMPIEZA DE COLUMNA RUT ===
Forma del DataFrame antes de limpiar: (927, 15)
Ejemplos de RUTs antes de limpiar:
['27042560-7', '27042560-7', '27042560-7', '15797997-3', '14194403-7']

Ejemplos de RUTs después de limpiar:
['270425607', '270425607', '270425607', '157979973', '141944037']
Forma del DataFrame después de limpiar: (927, 15)
RUTs que aún contienen guiones: 0


,Rut,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,270425607,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,15-07-2025,T,07:50,17:00,07:58:08,17:00:00 *,NaN,NaN,Olvido de Marcaje,No registra marcas en morpho,Solo salida
1,270425607,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,18-07-2025,D,07:50,14:30,08:00:23,14:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido de marca,Solo salida
2,270425607,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,05-08-2025,T,07:50,17:00,08:09:20,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido de marca,Solo salida
3,157979973,Débora Ester Muñoz Barrientos,Administración Gral De Producción,Ingeniero,07-07-2025,T,07:50,17:00,07:38:38,17:21:51,NaN,NaN,NaN,NaN,Entrada y salida
4,141944037,Manuel Jesus Abarca Moreira,Esencias y Env. Sabores,Operario,07-07-2025,T,07:50,17:00,07:50:00 *,16:55:31,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada


In [37]:
# Filtrar por motivos específicos de marca
print("\n=== FILTRADO POR MOTIVOS DE MARCA ===")
print(f"Forma del DataFrame antes del filtrado: {df_reporte_omisiones.shape}")

# Definir los motivos que queremos mantener
motivos_deseados = ["Olvido de Marcaje", "Otro"]

# Verificar qué columnas están disponibles para motivos
columnas_disponibles = df_reporte_omisiones.columns.tolist()
print(f"Columnas disponibles: {columnas_disponibles}")

# Buscar las columnas de motivos (pueden tener nombres ligeramente diferentes)
columna_entrada = None
columna_salida = None

for col in columnas_disponibles:
    if 'entrada' in col.lower() and 'motivo' in col.lower():
        columna_entrada = col
    elif 'salida' in col.lower() and 'motivo' in col.lower():
        columna_salida = col

print(f"Columna motivo entrada encontrada: {columna_entrada}")
print(f"Columna motivo salida encontrada: {columna_salida}")

# Mostrar valores únicos en las columnas de motivos antes del filtrado
if columna_entrada:
    print(f"\nValores únicos en {columna_entrada}:")
    print(df_reporte_omisiones[columna_entrada].value_counts())

if columna_salida:
    print(f"\nValores únicos en {columna_salida}:")
    print(df_reporte_omisiones[columna_salida].value_counts())

# Crear el filtro para mantener registros con los motivos deseados
filtro_condiciones = []

if columna_entrada:
    filtro_entrada = df_reporte_omisiones[columna_entrada].isin(motivos_deseados)
    filtro_condiciones.append(filtro_entrada)

if columna_salida:
    filtro_salida = df_reporte_omisiones[columna_salida].isin(motivos_deseados)
    filtro_condiciones.append(filtro_salida)

# Aplicar el filtro (OR entre las condiciones)
if filtro_condiciones:
    filtro_final = filtro_condiciones[0]
    for condicion in filtro_condiciones[1:]:
        filtro_final = filtro_final | condicion
    
    df_reporte_omisiones_filtrado = df_reporte_omisiones[filtro_final].copy()
    
    print(f"\n=== RESULTADO DEL FILTRADO ===")
    print(f"Registros antes del filtrado: {df_reporte_omisiones.shape[0]}")
    print(f"Registros después del filtrado: {df_reporte_omisiones_filtrado.shape[0]}")
    print(f"Registros eliminados: {df_reporte_omisiones.shape[0] - df_reporte_omisiones_filtrado.shape[0]}")
    
    # Mostrar distribución de motivos después del filtrado
    if columna_entrada:
        print(f"\nDistribución de {columna_entrada} después del filtrado:")
        print(df_reporte_omisiones_filtrado[columna_entrada].value_counts())
    
    if columna_salida:
        print(f"\nDistribución de {columna_salida} después del filtrado:")
        print(df_reporte_omisiones_filtrado[columna_salida].value_counts())
    
    # Reemplazar el dataframe original con el filtrado
    df_reporte_omisiones = df_reporte_omisiones_filtrado
    
    print(f"\n=== MUESTRA DEL DATAFRAME FILTRADO ===")
    display(df_reporte_omisiones.head())
    
else:
    print("No se encontraron columnas de motivos para filtrar")
    print("Columnas disponibles:", columnas_disponibles)


=== FILTRADO POR MOTIVOS DE MARCA ===
Forma del DataFrame antes del filtrado: (927, 15)
Columnas disponibles: ['Rut', 'Nombre', 'Unidad', 'Puesto', 'Fecha', 'Tipo de turno', 'Entrada programada', 'Salida programada', 'Marca de entrada', 'Marca de salida', 'Motivo marca de entrada', 'Comentario marca de entrada', 'Motivo marca de salida', 'Comentario marca de salida', 'Tipo de corrección']
Columna motivo entrada encontrada: Motivo marca de entrada
Columna motivo salida encontrada: Motivo marca de salida

Valores únicos en Motivo marca de entrada:
Motivo marca de entrada
Otro                 344
Olvido de Marcaje    183
Name: count, dtype: int64

Valores únicos en Motivo marca de salida:
Motivo marca de salida
Otro                 359
Olvido de Marcaje    342
Tiene marca            1
Name: count, dtype: int64

=== RESULTADO DEL FILTRADO ===
Registros antes del filtrado: 927
Registros después del filtrado: 851
Registros eliminados: 76

Distribución de Motivo marca de entrada después del 

,Rut,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,Motivo marca de entrada,Comentario marca de entrada,Motivo marca de salida,Comentario marca de salida,Tipo de corrección
0,270425607,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,15-07-2025,T,07:50,17:00,07:58:08,17:00:00 *,NaN,NaN,Olvido de Marcaje,No registra marcas en morpho,Solo salida
1,270425607,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,18-07-2025,D,07:50,14:30,08:00:23,14:30:00 *,NaN,NaN,Olvido de Marcaje,Olvido de marca,Solo salida
2,270425607,Lizeth García Ladrón De Guevara,Recursos Humanos,Jefatura,05-08-2025,T,07:50,17:00,08:09:20,17:00:00 *,NaN,NaN,Olvido de Marcaje,Olvido de marca,Solo salida
4,141944037,Manuel Jesus Abarca Moreira,Esencias y Env. Sabores,Operario,07-07-2025,T,07:50,17:00,07:50:00 *,16:55:31,Olvido de Marcaje,NaN,NaN,NaN,Solo entrada
5,141944037,Manuel Jesus Abarca Moreira,Esencias y Env. Sabores,Operario,10-07-2025,T,07:50,17:00,07:59:16,17:00:00 *,NaN,NaN,Olvido de Marcaje,NaN,Solo salida


In [38]:
# Cruce entre dataframes para contar omisiones de marcaje por periodo
print("\n=== CRUCE DE DATAFRAMES Y CONTEO DE OMISIONES DE MARCAJE ===")

# Verificar que tengamos datos filtrados
print(f"Registros en df_reporte_omisiones filtrado: {df_reporte_omisiones.shape[0]}")
print(f"Registros en df_filtrado_puestos: {df_filtrado_puestos.shape[0]}")

# Convertir la columna fecha en df_reporte_omisiones a datetime
if 'Fecha' in df_reporte_omisiones.columns:
    df_reporte_omisiones['Fecha'] = pd.to_datetime(df_reporte_omisiones['Fecha'])
    columna_fecha_omisiones = 'Fecha'
elif 'fecha' in df_reporte_omisiones.columns:
    df_reporte_omisiones['fecha'] = pd.to_datetime(df_reporte_omisiones['fecha'])
    columna_fecha_omisiones = 'fecha'
else:
    # Buscar columna de fecha
    for col in df_reporte_omisiones.columns:
        if 'fecha' in col.lower():
            df_reporte_omisiones[col] = pd.to_datetime(df_reporte_omisiones[col])
            columna_fecha_omisiones = col
            break
    else:
        print("No se encontró columna de fecha en df_reporte_omisiones")
        columna_fecha_omisiones = None

print(f"Columna de fecha utilizada: {columna_fecha_omisiones}")

# Definir los periodos (mismos que antes)
periodo_1_inicio = pd.to_datetime('2025-07-06')
periodo_1_fin = pd.to_datetime('2025-07-20')
periodo_2_inicio = pd.to_datetime('2025-07-21')
periodo_2_fin = pd.to_datetime('2025-08-05')

print(f"Periodo 1: {periodo_1_inicio.strftime('%d-%m-%Y')} al {periodo_1_fin.strftime('%d-%m-%Y')}")
print(f"Periodo 2: {periodo_2_inicio.strftime('%d-%m-%Y')} al {periodo_2_fin.strftime('%d-%m-%Y')}")

# Buscar las columnas de motivos ya identificadas anteriormente
columnas_disponibles = df_reporte_omisiones.columns.tolist()
columna_entrada = None
columna_salida = None

for col in columnas_disponibles:
    if 'entrada' in col.lower() and 'motivo' in col.lower():
        columna_entrada = col
    elif 'salida' in col.lower() and 'motivo' in col.lower():
        columna_salida = col

print(f"Columnas de motivos: Entrada={columna_entrada}, Salida={columna_salida}")

if columna_fecha_omisiones and (columna_entrada or columna_salida):
    
    # CONTEO PERIODO 1
    # Filtrar registros del periodo 1
    omisiones_p1 = df_reporte_omisiones[
        (df_reporte_omisiones[columna_fecha_omisiones] >= periodo_1_inicio) & 
        (df_reporte_omisiones[columna_fecha_omisiones] <= periodo_1_fin)
    ].copy()
    
    # Contar "Olvido de Marcaje" en entrada y salida por RUT
    conteo_entrada_p1 = pd.DataFrame()
    conteo_salida_p1 = pd.DataFrame()
    
    if columna_entrada:
        conteo_entrada_p1 = omisiones_p1[
            omisiones_p1[columna_entrada] == "Olvido de Marcaje"
        ].groupby('Rut').size().reset_index(name='Olvido_entrada_p1')
    
    if columna_salida:
        conteo_salida_p1 = omisiones_p1[
            omisiones_p1[columna_salida] == "Olvido de Marcaje"
        ].groupby('Rut').size().reset_index(name='Olvido_salida_p1')
    
    # CONTEO PERIODO 2
    # Filtrar registros del periodo 2
    omisiones_p2 = df_reporte_omisiones[
        (df_reporte_omisiones[columna_fecha_omisiones] >= periodo_2_inicio) & 
        (df_reporte_omisiones[columna_fecha_omisiones] <= periodo_2_fin)
    ].copy()
    
    # Contar "Olvido de Marcaje" en entrada y salida por RUT
    conteo_entrada_p2 = pd.DataFrame()
    conteo_salida_p2 = pd.DataFrame()
    
    if columna_entrada:
        conteo_entrada_p2 = omisiones_p2[
            omisiones_p2[columna_entrada] == "Olvido de Marcaje"
        ].groupby('Rut').size().reset_index(name='Olvido_entrada_p2')
    
    if columna_salida:
        conteo_salida_p2 = omisiones_p2[
            omisiones_p2[columna_salida] == "Olvido de Marcaje"
        ].groupby('Rut').size().reset_index(name='Olvido_salida_p2')
    
    # Crear dataframe consolidado con todos los conteos
    ruts_unicos = df_filtrado_puestos['rut'].unique()
    df_omisiones_marcaje = pd.DataFrame({'rut': ruts_unicos})
    
    # Hacer merge con todos los conteos (renombrar Rut a rut para compatibilidad)
    if not conteo_entrada_p1.empty:
        conteo_entrada_p1['rut'] = conteo_entrada_p1['Rut']
        df_omisiones_marcaje = df_omisiones_marcaje.merge(conteo_entrada_p1[['rut', 'Olvido_entrada_p1']], on='rut', how='left')
    
    if not conteo_salida_p1.empty:
        conteo_salida_p1['rut'] = conteo_salida_p1['Rut']
        df_omisiones_marcaje = df_omisiones_marcaje.merge(conteo_salida_p1[['rut', 'Olvido_salida_p1']], on='rut', how='left')
    
    if not conteo_entrada_p2.empty:
        conteo_entrada_p2['rut'] = conteo_entrada_p2['Rut']
        df_omisiones_marcaje = df_omisiones_marcaje.merge(conteo_entrada_p2[['rut', 'Olvido_entrada_p2']], on='rut', how='left')
    
    if not conteo_salida_p2.empty:
        conteo_salida_p2['rut'] = conteo_salida_p2['Rut']
        df_omisiones_marcaje = df_omisiones_marcaje.merge(conteo_salida_p2[['rut', 'Olvido_salida_p2']], on='rut', how='left')
    
    # Rellenar valores nulos con 0
    columnas_olvido = ['Olvido_entrada_p1', 'Olvido_salida_p1', 'Olvido_entrada_p2', 'Olvido_salida_p2']
    for col in columnas_olvido:
        if col in df_omisiones_marcaje.columns:
            df_omisiones_marcaje[col] = df_omisiones_marcaje[col].fillna(0).astype(int)
        else:
            df_omisiones_marcaje[col] = 0
    
    # Crear columnas totales por periodo
    df_omisiones_marcaje['Olvidos_marcaje_periodo_1'] = (
        df_omisiones_marcaje['Olvido_entrada_p1'] + df_omisiones_marcaje['Olvido_salida_p1']
    )
    df_omisiones_marcaje['Olvidos_marcaje_periodo_2'] = (
        df_omisiones_marcaje['Olvido_entrada_p2'] + df_omisiones_marcaje['Olvido_salida_p2']
    )
    
    # Hacer merge con el dataframe principal
    columnas_a_agregar = ['Olvido_entrada_p1', 'Olvido_salida_p1', 'Olvido_entrada_p2', 'Olvido_salida_p2', 
                         'Olvidos_marcaje_periodo_1', 'Olvidos_marcaje_periodo_2']
    
    df_filtrado_puestos = df_filtrado_puestos.merge(
        df_omisiones_marcaje[['rut'] + columnas_a_agregar], 
        on='rut', 
        how='left'
    )
    
    # Rellenar valores nulos en el dataframe principal
    for col in columnas_a_agregar:
        df_filtrado_puestos[col] = df_filtrado_puestos[col].fillna(0).astype(int)
    
    print(f"\n=== RESUMEN DE OLVIDOS DE MARCAJE ===")
    print(f"PERIODO 1:")
    print(f"  Trabajadores con olvidos de entrada: {(df_omisiones_marcaje['Olvido_entrada_p1'] > 0).sum()}")
    print(f"  Trabajadores con olvidos de salida: {(df_omisiones_marcaje['Olvido_salida_p1'] > 0).sum()}")
    print(f"  Total olvidos entrada: {df_omisiones_marcaje['Olvido_entrada_p1'].sum()}")
    print(f"  Total olvidos salida: {df_omisiones_marcaje['Olvido_salida_p1'].sum()}")
    print(f"  Total olvidos periodo 1: {df_omisiones_marcaje['Olvidos_marcaje_periodo_1'].sum()}")
    
    print(f"\nPERIODO 2:")
    print(f"  Trabajadores con olvidos de entrada: {(df_omisiones_marcaje['Olvido_entrada_p2'] > 0).sum()}")
    print(f"  Trabajadores con olvidos de salida: {(df_omisiones_marcaje['Olvido_salida_p2'] > 0).sum()}")
    print(f"  Total olvidos entrada: {df_omisiones_marcaje['Olvido_entrada_p2'].sum()}")
    print(f"  Total olvidos salida: {df_omisiones_marcaje['Olvido_salida_p2'].sum()}")
    print(f"  Total olvidos periodo 2: {df_omisiones_marcaje['Olvidos_marcaje_periodo_2'].sum()}")
    
    print(f"\n=== COLUMNAS AGREGADAS AL DATAFRAME PRINCIPAL ===")
    print(f"- Olvido_entrada_p1: Olvidos de marcaje de entrada en periodo 1")
    print(f"- Olvido_salida_p1: Olvidos de marcaje de salida en periodo 1")
    print(f"- Olvido_entrada_p2: Olvidos de marcaje de entrada en periodo 2")
    print(f"- Olvido_salida_p2: Olvidos de marcaje de salida en periodo 2")
    print(f"- Olvidos_marcaje_periodo_1: Total olvidos (entrada + salida) periodo 1")
    print(f"- Olvidos_marcaje_periodo_2: Total olvidos (entrada + salida) periodo 2")
    
    # Mostrar muestra de los resultados
    columnas_muestra = ['rut', 'nombre', 'PuestoIntegracion_trabajador'] + columnas_a_agregar
    print(f"\n=== MUESTRA DE RESULTADOS ===")
    display(df_filtrado_puestos[columnas_muestra].drop_duplicates('rut').head(10))
    
else:
    print("No se pudo realizar el cruce por falta de columnas necesarias")
    if not columna_fecha_omisiones:
        print("- Falta columna de fecha")
    if not columna_entrada and not columna_salida:
        print("- Faltan columnas de motivos")


=== CRUCE DE DATAFRAMES Y CONTEO DE OMISIONES DE MARCAJE ===
Registros en df_reporte_omisiones filtrado: 851
Registros en df_filtrado_puestos: 5835
Columna de fecha utilizada: Fecha
Periodo 1: 06-07-2025 al 20-07-2025
Periodo 2: 21-07-2025 al 05-08-2025
Columnas de motivos: Entrada=Motivo marca de entrada, Salida=Motivo marca de salida

=== RESUMEN DE OLVIDOS DE MARCAJE ===
PERIODO 1:
  Trabajadores con olvidos de entrada: 2
  Trabajadores con olvidos de salida: 3
  Total olvidos entrada: 2
  Total olvidos salida: 3
  Total olvidos periodo 1: 5

PERIODO 2:
  Trabajadores con olvidos de entrada: 2
  Trabajadores con olvidos de salida: 2
  Total olvidos entrada: 2
  Total olvidos salida: 2
  Total olvidos periodo 2: 4

=== COLUMNAS AGREGADAS AL DATAFRAME PRINCIPAL ===
- Olvido_entrada_p1: Olvidos de marcaje de entrada en periodo 1
- Olvido_salida_p1: Olvidos de marcaje de salida en periodo 1
- Olvido_entrada_p2: Olvidos de marcaje de entrada en periodo 2
- Olvido_salida_p2: Olvidos de m

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_9868\4250272927.py:10: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_reporte_omisiones['Fecha'] = pd.to_datetime(df_reporte_omisiones['Fecha'])


,rut,nombre,PuestoIntegracion_trabajador,Olvido_entrada_p1,Olvido_salida_p1,Olvido_entrada_p2,Olvido_salida_p2,Olvidos_marcaje_periodo_1,Olvidos_marcaje_periodo_2
0,128294988,Cesar,Asistente De Servicios Generales,0,0,0,0,0,0
21,256248786,Marie,Asistente De Servicios Generales,0,0,0,0,0,0
26,107806466,Rosa,Asistente De Servicios Generales,0,0,0,0,0,0
31,82755640,Marcela,Asistente De Servicios Generales,0,0,0,0,0,0
37,116207923,Rosa,Asistente De Servicios Generales,0,0,0,0,0,0
108,261696436,Eyver,Operario,0,0,0,0,0,0
129,158229838,Carlos,Operario,0,0,0,0,0,0
152,268128743,Renato,Inspector De Proceso,0,0,0,0,0,0
173,158929724,Maritza,Inspector De Proceso,0,0,0,0,0,0
194,270363067,Jesus,Inspector De Proceso,0,0,0,0,0,0


In [39]:
# Exportar dataframe con todas las incidencias por periodo a Excel
print("=== EXPORTACIÓN A EXCEL ===")

# Crear un dataframe único por trabajador con todas las columnas de incidencias
columnas_exportar = [
    'rut', 
    'nombre', 
    'apellidoPaterno', 
    'apellidoMaterno',
    'PuestoIntegracion_trabajador',
    'Centrocosto_trabajador',
    # Licencias
    'Licencia_1ra_qcna',
    'Licencias_periodo_1', 
    'Licencias_periodo_2',
    # Permisos
    'Permisos_periodo_1', 
    'Permisos_periodo_2',
    # Ausencias injustificadas
    'Ausencias_injust_periodo_1', 
    'Ausencias_injust_periodo_2',
    # Atrasos
    'Atrasos_periodo_1', 
    'Atrasos_periodo_2',
    # Omisiones detectadas automáticamente
    'Omisiones_periodo_1', 
    'Omisiones_periodo_2',
    # Olvidos de marcaje del reporte externo
    'Olvido_entrada_p1', 
    'Olvido_salida_p1', 
    'Olvido_entrada_p2', 
    'Olvido_salida_p2',
    'Olvidos_marcaje_periodo_1', 
    'Olvidos_marcaje_periodo_2'
]

# Crear dataframe único por trabajador
df_export = df_filtrado_puestos[columnas_exportar].drop_duplicates('rut').copy()

# Ordenar por nombre para mejor presentación
df_export = df_export.sort_values(['apellidoPaterno', 'apellidoMaterno', 'nombre'])

# Crear nombre de archivo con timestamp
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
nombre_archivo = f"Reporte_Bono_Asistencia_COMPLETO_{timestamp}.xlsx"
ruta_archivo = rf"C:\Users\bgacitua\Desktop\Scripts de Python\Reportes generados\{nombre_archivo}"

# Exportar a Excel
df_export.to_excel(ruta_archivo, index=False, sheet_name='Reporte_Incidencias')

print(f"Archivo exportado exitosamente:")
print(f"Ruta: {ruta_archivo}")
print(f"Nombre: {nombre_archivo}")
print(f"Registros exportados: {len(df_export)}")
print(f"Columnas exportadas: {len(columnas_exportar)}")

print(f"\n=== RESUMEN DEL ARCHIVO EXPORTADO ===")
print(f"Trabajadores únicos: {df_export['rut'].nunique()}")
print(f"Puestos únicos: {df_export['PuestoIntegracion_trabajador'].nunique()}")
print(f"Centros de costo únicos: {df_export['Centrocosto_trabajador'].nunique()}")

print(f"\n=== ESTADÍSTICAS GENERALES ===")
print(f"Total licencias período 1: {df_export['Licencias_periodo_1'].sum()}")
print(f"Total licencias período 2: {df_export['Licencias_periodo_2'].sum()}")
print(f"Total permisos período 1: {df_export['Permisos_periodo_1'].sum()}")
print(f"Total permisos período 2: {df_export['Permisos_periodo_2'].sum()}")
print(f"Total ausencias injustificadas período 1: {df_export['Ausencias_injust_periodo_1'].sum()}")
print(f"Total ausencias injustificadas período 2: {df_export['Ausencias_injust_periodo_2'].sum()}")
print(f"Total atrasos período 1: {df_export['Atrasos_periodo_1'].sum()}")
print(f"Total atrasos período 2: {df_export['Atrasos_periodo_2'].sum()}")
print(f"Total omisiones detectadas período 1: {df_export['Omisiones_periodo_1'].sum()}")
print(f"Total omisiones detectadas período 2: {df_export['Omisiones_periodo_2'].sum()}")
print(f"Total olvidos marcaje período 1: {df_export['Olvidos_marcaje_periodo_1'].sum()}")
print(f"Total olvidos marcaje período 2: {df_export['Olvidos_marcaje_periodo_2'].sum()}")

print(f"\n=== VISTA PREVIA DEL ARCHIVO ===")
display(df_export.head(10))

=== EXPORTACIÓN A EXCEL ===
Archivo exportado exitosamente:
Ruta: C:\Users\bgacitua\Desktop\Scripts de Python\Reportes generados\Reporte_Bono_Asistencia_COMPLETO_20250813_101630.xlsx
Nombre: Reporte_Bono_Asistencia_COMPLETO_20250813_101630.xlsx
Registros exportados: 283
Columnas exportadas: 23

=== RESUMEN DEL ARCHIVO EXPORTADO ===
Trabajadores únicos: 283
Puestos únicos: 17
Centros de costo únicos: 23

=== ESTADÍSTICAS GENERALES ===
Total licencias período 1: 78
Total licencias período 2: 111
Total permisos período 1: 3
Total permisos período 2: 10
Total ausencias injustificadas período 1: 0
Total ausencias injustificadas período 2: 6
Total atrasos período 1: 106
Total atrasos período 2: 154
Total omisiones detectadas período 1: 3
Total omisiones detectadas período 2: 0
Total olvidos marcaje período 1: 5
Total olvidos marcaje período 2: 4

=== VISTA PREVIA DEL ARCHIVO ===
Archivo exportado exitosamente:
Ruta: C:\Users\bgacitua\Desktop\Scripts de Python\Reportes generados\Reporte_Bono_

,rut,nombre,apellidoPaterno,apellidoMaterno,PuestoIntegracion_trabajador,Centrocosto_trabajador,Licencia_1ra_qcna,Licencias_periodo_1,Licencias_periodo_2,Permisos_periodo_1,...,Atrasos_periodo_1,Atrasos_periodo_2,Omisiones_periodo_1,Omisiones_periodo_2,Olvido_entrada_p1,Olvido_salida_p1,Olvido_entrada_p2,Olvido_salida_p2,Olvidos_marcaje_periodo_1,Olvidos_marcaje_periodo_2
3259,123577914,Cristian,Abarca,Garrido,Operario,Fabricación Y Envasado Mezclas Polvos,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1982,141944037,Manuel,Abarca,Moreira,Operario,Envasado Bodega Sabores,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
5562,197927445,Javiera,Abarca,Riveros,Asistente De Laboratorio,Desarrollo Sabores Dulces,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4076,201626455,Daniel,Adonis,Adones,Asistente De Laboratorio,Muestras Sabores,0,0,0,0,...,1,5,0,0,0,0,0,0,0,0
3492,122919951,Hugo,Adrian,Medel,Operario,Fabricación Y Envasado Mezclas Polvos,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
441,86069911,Flaminio,Agurto,Urrutia,Chofer,Transporte,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
693,207290149,Javier,Alarcon,Andrade,Ayudante De Bodega,Despacho,0,0,0,0,...,2,1,0,0,0,0,0,0,0,0
5583,192922321,Francisca,Alarcon,Madariaga,Asistente De Laboratorio,Desarrollo Sabores Bebidas,0,0,0,0,...,1,3,0,0,0,0,0,0,0,0
2440,17390102K,Mario,Alvarado,Bravo,Operario,Envasado Bodega Sabores,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2698,170291247,Jose,Alvarado,Dip,Operario,Envasado Bodega Sabores,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# CORRECCIÓN MEJORADA: Limpiar RUTs con dígito verificador
print("=== CORRECCIÓN MEJORADA - LIMPIEZA DE RUTS ===")

print("1. ANALIZAR TIPOS DE RUT EN df_reporte_omisiones:")
print("Ejemplos de RUTs en df_reporte_omisiones:")
print(df_reporte_omisiones['Rut'].head(10).tolist())

# Verificar si hay RUTs con 'K' o otros caracteres
ruts_con_k = df_reporte_omisiones['Rut'].astype(str).str.contains('K', na=False).sum()
print(f"RUTs con 'K': {ruts_con_k}")

print("\n2. LIMPIAR RUTS ELIMINANDO DÍGITO VERIFICADOR:")
# Crear función para limpiar RUTs
def limpiar_rut(rut):
    """
    Limpia un RUT eliminando el dígito verificador y caracteres no numéricos
    """
    if pd.isna(rut):
        return None
    
    # Convertir a string y limpiar
    rut_str = str(rut).strip()
    
    # Eliminar caracteres no numéricos excepto números
    rut_limpio = ''.join(filter(str.isdigit, rut_str))
    
    # Si queda vacío, retornar None
    if not rut_limpio:
        return None
    
    try:
        return int(rut_limpio)
    except:
        return None

# Aplicar limpieza a la columna de RUT
print("Limpiando RUTs en df_reporte_omisiones...")
df_reporte_omisiones['Rut_limpio'] = df_reporte_omisiones['Rut'].apply(limpiar_rut)

# Verificar resultado de la limpieza
print("Ejemplos de RUTs después de limpiar:")
muestra_limpieza = df_reporte_omisiones[['Rut', 'Rut_limpio']].head(10)
display(muestra_limpieza)

# Contar RUTs válidos después de limpieza
ruts_validos = df_reporte_omisiones['Rut_limpio'].notna().sum()
total_ruts = len(df_reporte_omisiones)
print(f"RUTs válidos después de limpieza: {ruts_validos} de {total_ruts}")

# Verificar si nuestro RUT problemático está bien después de la limpieza
rut_problema_en_limpio = df_reporte_omisiones[df_reporte_omisiones['Rut_limpio'] == 141944037]
print(f"\nRUT 141944037 encontrado después de limpieza: {len(rut_problema_en_limpio)} registros")

if len(rut_problema_en_limpio) > 0:
    print("Información del RUT problemático después de limpieza:")
    display(rut_problema_en_limpio[['Rut', 'Rut_limpio', 'Nombre', 'Fecha']])

print("\n3. REPETIR CRUCE CON RUTS LIMPIOS:")

# Usar la columna Rut_limpio para el cruce
if len(rut_problema_en_limpio) > 0:
    
    # CONTEO PERIODO 1 - CON RUTS LIMPIOS
    omisiones_p1_limpio = df_reporte_omisiones[
        (df_reporte_omisiones[columna_fecha_omisiones] >= periodo_1_inicio) & 
        (df_reporte_omisiones[columna_fecha_omisiones] <= periodo_1_fin) &
        (df_reporte_omisiones['Rut_limpio'].notna())
    ].copy()
    
    # Contar "Olvido de Marcaje" en entrada y salida por RUT LIMPIO
    conteo_entrada_p1_limpio = pd.DataFrame()
    conteo_salida_p1_limpio = pd.DataFrame()
    
    if columna_entrada:
        conteo_entrada_p1_limpio = omisiones_p1_limpio[
            omisiones_p1_limpio[columna_entrada] == "Olvido de Marcaje"
        ].groupby('Rut_limpio').size().reset_index(name='Olvido_entrada_p1')
    
    if columna_salida:
        conteo_salida_p1_limpio = omisiones_p1_limpio[
            omisiones_p1_limpio[columna_salida] == "Olvido de Marcaje"
        ].groupby('Rut_limpio').size().reset_index(name='Olvido_salida_p1')
    
    # CONTEO PERIODO 2 - CON RUTS LIMPIOS
    omisiones_p2_limpio = df_reporte_omisiones[
        (df_reporte_omisiones[columna_fecha_omisiones] >= periodo_2_inicio) & 
        (df_reporte_omisiones[columna_fecha_omisiones] <= periodo_2_fin) &
        (df_reporte_omisiones['Rut_limpio'].notna())
    ].copy()
    
    conteo_entrada_p2_limpio = pd.DataFrame()
    conteo_salida_p2_limpio = pd.DataFrame()
    
    if columna_entrada:
        conteo_entrada_p2_limpio = omisiones_p2_limpio[
            omisiones_p2_limpio[columna_entrada] == "Olvido de Marcaje"
        ].groupby('Rut_limpio').size().reset_index(name='Olvido_entrada_p2')
    
    if columna_salida:
        conteo_salida_p2_limpio = omisiones_p2_limpio[
            omisiones_p2_limpio[columna_salida] == "Olvido de Marcaje"
        ].groupby('Rut_limpio').size().reset_index(name='Olvido_salida_p2')
    
    print(f"Conteos por periodo:")
    print(f"  Entrada P1: {len(conteo_entrada_p1_limpio)} registros")
    print(f"  Salida P1: {len(conteo_salida_p1_limpio)} registros")
    print(f"  Entrada P2: {len(conteo_entrada_p2_limpio)} registros")
    print(f"  Salida P2: {len(conteo_salida_p2_limpio)} registros")
    
    # Verificar específicamente nuestro RUT problemático
    if not conteo_salida_p1_limpio.empty:
        rut_en_conteo = conteo_salida_p1_limpio[conteo_salida_p1_limpio['Rut_limpio'] == 141944037]
        print(f"\nRUT 141944037 en conteo_salida_p1: {len(rut_en_conteo)} registros")
        if len(rut_en_conteo) > 0:
            display(rut_en_conteo)
    
    print(f"\n=== RESULTADO DE LA CORRECCIÓN ===")

=== CORRECCIÓN MEJORADA - LIMPIEZA DE RUTS ===
1. ANALIZAR TIPOS DE RUT EN df_reporte_omisiones:
Ejemplos de RUTs en df_reporte_omisiones:
['270425607', '270425607', '270425607', '141944037', '141944037', '141944037', '141944037', '90338331', '90338331', '90338331']
RUTs con 'K': 51

2. LIMPIAR RUTS ELIMINANDO DÍGITO VERIFICADOR:
Limpiando RUTs en df_reporte_omisiones...
Ejemplos de RUTs después de limpiar:

1. ANALIZAR TIPOS DE RUT EN df_reporte_omisiones:
Ejemplos de RUTs en df_reporte_omisiones:
['270425607', '270425607', '270425607', '141944037', '141944037', '141944037', '141944037', '90338331', '90338331', '90338331']
RUTs con 'K': 51

2. LIMPIAR RUTS ELIMINANDO DÍGITO VERIFICADOR:
Limpiando RUTs en df_reporte_omisiones...
Ejemplos de RUTs después de limpiar:


,Rut,Rut_limpio
0,270425607,270425607
1,270425607,270425607
2,270425607,270425607
4,141944037,141944037
5,141944037,141944037
6,141944037,141944037
7,141944037,141944037
8,90338331,90338331
9,90338331,90338331
10,90338331,90338331


RUTs válidos después de limpieza: 851 de 851

RUT 141944037 encontrado después de limpieza: 4 registros
Información del RUT problemático después de limpieza:


,Rut,Rut_limpio,Nombre,Fecha
4,141944037,141944037,Manuel Jesus Abarca Moreira,2025-07-07
5,141944037,141944037,Manuel Jesus Abarca Moreira,2025-07-10
6,141944037,141944037,Manuel Jesus Abarca Moreira,2025-07-24
7,141944037,141944037,Manuel Jesus Abarca Moreira,2025-07-25



3. REPETIR CRUCE CON RUTS LIMPIOS:
Conteos por periodo:
  Entrada P1: 50 registros
  Salida P1: 82 registros
  Entrada P2: 77 registros
  Salida P2: 103 registros

RUT 141944037 en conteo_salida_p1: 1 registros


,Rut_limpio,Olvido_salida_p1
31,141944037,1



=== RESULTADO DE LA CORRECCIÓN ===


In [41]:
# APLICAR CORRECCIÓN COMPLETA AL DATAFRAME PRINCIPAL
print("=== APLICAR CORRECCIÓN COMPLETA ===")

# Crear dataframe consolidado con RUTs limpios
ruts_unicos = df_filtrado_puestos['rut'].unique()
df_omisiones_marcaje_final = pd.DataFrame({'rut': ruts_unicos})

# Hacer merge con todos los conteos usando RUT limpio
if not conteo_entrada_p1_limpio.empty:
    conteo_entrada_p1_limpio.rename(columns={'Rut_limpio': 'rut'}, inplace=True)
    df_omisiones_marcaje_final = df_omisiones_marcaje_final.merge(
        conteo_entrada_p1_limpio[['rut', 'Olvido_entrada_p1']], on='rut', how='left')

if not conteo_salida_p1_limpio.empty:
    conteo_salida_p1_limpio.rename(columns={'Rut_limpio': 'rut'}, inplace=True)
    df_omisiones_marcaje_final = df_omisiones_marcaje_final.merge(
        conteo_salida_p1_limpio[['rut', 'Olvido_salida_p1']], on='rut', how='left')

if not conteo_entrada_p2_limpio.empty:
    conteo_entrada_p2_limpio.rename(columns={'Rut_limpio': 'rut'}, inplace=True)
    df_omisiones_marcaje_final = df_omisiones_marcaje_final.merge(
        conteo_entrada_p2_limpio[['rut', 'Olvido_entrada_p2']], on='rut', how='left')

if not conteo_salida_p2_limpio.empty:
    conteo_salida_p2_limpio.rename(columns={'Rut_limpio': 'rut'}, inplace=True)
    df_omisiones_marcaje_final = df_omisiones_marcaje_final.merge(
        conteo_salida_p2_limpio[['rut', 'Olvido_salida_p2']], on='rut', how='left')

# Rellenar valores nulos con 0
columnas_olvido_final = ['Olvido_entrada_p1', 'Olvido_salida_p1', 'Olvido_entrada_p2', 'Olvido_salida_p2']
for col in columnas_olvido_final:
    if col in df_omisiones_marcaje_final.columns:
        df_omisiones_marcaje_final[col] = df_omisiones_marcaje_final[col].fillna(0).astype(int)
    else:
        df_omisiones_marcaje_final[col] = 0

# Crear columnas totales por periodo
df_omisiones_marcaje_final['Olvidos_marcaje_periodo_1'] = (
    df_omisiones_marcaje_final['Olvido_entrada_p1'] + df_omisiones_marcaje_final['Olvido_salida_p1']
)
df_omisiones_marcaje_final['Olvidos_marcaje_periodo_2'] = (
    df_omisiones_marcaje_final['Olvido_entrada_p2'] + df_omisiones_marcaje_final['Olvido_salida_p2']
)

print("1. VERIFICAR RESULTADO ANTES DE APLICAR:")
rut_test = df_omisiones_marcaje_final[df_omisiones_marcaje_final['rut'] == 141944037]
print(f"RUT 141944037 en resultado final: {len(rut_test)} registros")
if len(rut_test) > 0:
    print("Valores finales para RUT 141944037:")
    display(rut_test)

print("\n2. ELIMINAR COLUMNAS INCORRECTAS DEL DATAFRAME PRINCIPAL:")
columnas_a_reemplazar = ['Olvido_entrada_p1', 'Olvido_salida_p1', 'Olvido_entrada_p2', 'Olvido_salida_p2',
                        'Olvidos_marcaje_periodo_1', 'Olvidos_marcaje_periodo_2']

for col in columnas_a_reemplazar:
    if col in df_filtrado_puestos.columns:
        df_filtrado_puestos.drop(col, axis=1, inplace=True)
        print(f"  Eliminada columna: {col}")

print("\n3. APLICAR MERGE CORREGIDO:")
df_filtrado_puestos = df_filtrado_puestos.merge(
    df_omisiones_marcaje_final[['rut'] + columnas_a_reemplazar], 
    on='rut', 
    how='left'
)

# Rellenar valores nulos
for col in columnas_a_reemplazar:
    df_filtrado_puestos[col] = df_filtrado_puestos[col].fillna(0).astype(int)

print("¡MERGE CORREGIDO APLICADO!")

print("\n4. VERIFICACIÓN FINAL:")
resultado_final = df_filtrado_puestos[df_filtrado_puestos['rut'] == 141944037]
if len(resultado_final) > 0:
    columnas_verificacion = ['rut', 'nombre', 'PuestoIntegracion_trabajador'] + columnas_a_reemplazar
    print("Resultado final para RUT 141944037:")
    display(resultado_final[columnas_verificacion].drop_duplicates('rut'))
else:
    print("ERROR: RUT 141944037 no encontrado en el dataframe final")

print("\n5. RESUMEN DE LA CORRECCIÓN:")
print(f"Total olvidos marcaje período 1: {df_omisiones_marcaje_final['Olvidos_marcaje_periodo_1'].sum()}")
print(f"Total olvidos marcaje período 2: {df_omisiones_marcaje_final['Olvidos_marcaje_periodo_2'].sum()}")
print(f"Trabajadores con olvidos P1: {(df_omisiones_marcaje_final['Olvidos_marcaje_periodo_1'] > 0).sum()}")
print(f"Trabajadores con olvidos P2: {(df_omisiones_marcaje_final['Olvidos_marcaje_periodo_2'] > 0).sum()}")

print(f"\n=== CORRECCIÓN COMPLETADA EXITOSAMENTE ===")

=== APLICAR CORRECCIÓN COMPLETA ===
1. VERIFICAR RESULTADO ANTES DE APLICAR:
RUT 141944037 en resultado final: 1 registros
Valores finales para RUT 141944037:
1. VERIFICAR RESULTADO ANTES DE APLICAR:
RUT 141944037 en resultado final: 1 registros
Valores finales para RUT 141944037:


,rut,Olvido_entrada_p1,Olvido_salida_p1,Olvido_entrada_p2,Olvido_salida_p2,Olvidos_marcaje_periodo_1,Olvidos_marcaje_periodo_2
94,141944037,1,1,2,0,2,2



2. ELIMINAR COLUMNAS INCORRECTAS DEL DATAFRAME PRINCIPAL:
  Eliminada columna: Olvido_entrada_p1
  Eliminada columna: Olvido_salida_p1
  Eliminada columna: Olvido_entrada_p2
  Eliminada columna: Olvido_salida_p2
  Eliminada columna: Olvidos_marcaje_periodo_1
  Eliminada columna: Olvidos_marcaje_periodo_2

3. APLICAR MERGE CORREGIDO:
¡MERGE CORREGIDO APLICADO!

4. VERIFICACIÓN FINAL:
Resultado final para RUT 141944037:


,rut,nombre,PuestoIntegracion_trabajador,Olvido_entrada_p1,Olvido_salida_p1,Olvido_entrada_p2,Olvido_salida_p2,Olvidos_marcaje_periodo_1,Olvidos_marcaje_periodo_2
1982,141944037,Manuel,Operario,1,1,2,0,2,2



5. RESUMEN DE LA CORRECCIÓN:
Total olvidos marcaje período 1: 28
Total olvidos marcaje período 2: 38
Trabajadores con olvidos P1: 24
Trabajadores con olvidos P2: 32

=== CORRECCIÓN COMPLETADA EXITOSAMENTE ===


In [42]:
# GENERAR ARCHIVO EXCEL CORREGIDO
from datetime import datetime

print("=== GENERANDO EXCEL CORREGIDO ===")

# Preparar datos para exportar (sin duplicados por trabajador)
df_para_exportar = df_filtrado_puestos.drop_duplicates(subset='rut')

# Seleccionar columnas para el reporte final
columnas_finales = [
    'rut', 'nombre', 'PuestoIntegracion_trabajador',
    # Licencias
    'licencias_medicas_periodo_1', 'licencias_medicas_periodo_2',
    # Permisos
    'permisos_periodo_1', 'permisos_periodo_2',
    # Ausencias
    'ausencias_injustificadas_periodo_1', 'ausencias_injustificadas_periodo_2',
    # Tardanzas (días y minutos)
    'dias_tardanza_periodo_1', 'minutos_tardanza_periodo_1',
    'dias_tardanza_periodo_2', 'minutos_tardanza_periodo_2',
    # Olvidos de marcaje (CORREGIDOS)
    'Olvido_entrada_p1', 'Olvido_salida_p1', 'Olvido_entrada_p2', 'Olvido_salida_p2',
    'Olvidos_marcaje_periodo_1', 'Olvidos_marcaje_periodo_2'
]

# Verificar que todas las columnas existen
columnas_existentes = [col for col in columnas_finales if col in df_para_exportar.columns]
columnas_faltantes = [col for col in columnas_finales if col not in df_para_exportar.columns]

if columnas_faltantes:
    print(f" Columnas faltantes: {columnas_faltantes}")

df_exportar = df_para_exportar[columnas_existentes].copy()

# Generar nombre de archivo con timestamp
timestamp = datetime.now().strftime("%d%m%Y_%H%M%S")
nombre_archivo = f"Reporte_Bono_Asistencia_CORREGIDO_{timestamp}.xlsx"
ruta_archivo = f"c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/{nombre_archivo}"

# Exportar a Excel
df_exportar.to_excel(ruta_archivo, index=False)

print(f"✅ Archivo Excel CORREGIDO generado exitosamente:")
print(f"📁 {ruta_archivo}")
print(f"📊 Registros exportados: {len(df_exportar)}")


=== GENERANDO EXCEL CORREGIDO ===
 Columnas faltantes: ['licencias_medicas_periodo_1', 'licencias_medicas_periodo_2', 'permisos_periodo_1', 'permisos_periodo_2', 'ausencias_injustificadas_periodo_1', 'ausencias_injustificadas_periodo_2', 'dias_tardanza_periodo_1', 'minutos_tardanza_periodo_1', 'dias_tardanza_periodo_2', 'minutos_tardanza_periodo_2']
✅ Archivo Excel CORREGIDO generado exitosamente:
📁 c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/Reporte_Bono_Asistencia_CORREGIDO_13082025_101630.xlsx
📊 Registros exportados: 283
✅ Archivo Excel CORREGIDO generado exitosamente:
📁 c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/Reporte_Bono_Asistencia_CORREGIDO_13082025_101630.xlsx
📊 Registros exportados: 283


In [43]:
# GENERAR EXCEL COMPLETO CON TODAS LAS COLUMNAS CALCULADAS
from datetime import datetime

print("=== VERIFICAR COLUMNAS DISPONIBLES ===")

# Mostrar todas las columnas del dataframe principal
print("Columnas disponibles en df_filtrado_puestos:")
for i, col in enumerate(df_filtrado_puestos.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nTotal de columnas: {len(df_filtrado_puestos.columns)}")
print(f"Total de registros: {len(df_filtrado_puestos)}")
print(f"RUTs únicos: {df_filtrado_puestos['rut'].nunique()}")

# Preparar datos para exportar (sin duplicados por trabajador)
df_para_exportar_completo = df_filtrado_puestos.drop_duplicates(subset='rut')

print(f"\nRegistros después de eliminar duplicados: {len(df_para_exportar_completo)}")

# Organizar columnas por categorías para mejor orden en el Excel
columnas_identificacion = ['rut', 'nombre', 'PuestoIntegracion_trabajador']

# Buscar columnas de licencias
columnas_licencias = [col for col in df_para_exportar_completo.columns 
                     if 'licencia' in col.lower() or 'licencia' in col]

# Buscar columnas de permisos  
columnas_permisos = [col for col in df_para_exportar_completo.columns 
                    if 'permiso' in col.lower()]

# Buscar columnas de ausencias
columnas_ausencias = [col for col in df_para_exportar_completo.columns 
                     if 'ausencia' in col.lower()]

# Buscar columnas de tardanzas/atrasos
columnas_tardanzas = [col for col in df_para_exportar_completo.columns 
                     if 'tardanza' in col.lower() or 'atraso' in col.lower() or 'minutos' in col.lower()]

# Buscar columnas de olvidos/omisiones
columnas_olvidos = [col for col in df_para_exportar_completo.columns 
                   if 'olvido' in col.lower() or 'omision' in col.lower()]

print("\n=== ORGANIZACIÓN DE COLUMNAS ===")
print(f"Identificación ({len(columnas_identificacion)}): {columnas_identificacion}")
print(f"Licencias ({len(columnas_licencias)}): {columnas_licencias}")
print(f"Permisos ({len(columnas_permisos)}): {columnas_permisos}")
print(f"Ausencias ({len(columnas_ausencias)}): {columnas_ausencias}")
print(f"Tardanzas ({len(columnas_tardanzas)}): {columnas_tardanzas}")
print(f"Olvidos ({len(columnas_olvidos)}): {columnas_olvidos}")

# Ordenar columnas de manera lógica
columnas_ordenadas = (columnas_identificacion + 
                     columnas_licencias + 
                     columnas_permisos + 
                     columnas_ausencias + 
                     columnas_tardanzas + 
                     columnas_olvidos)

# Agregar cualquier columna que no hayamos categorizado
columnas_restantes = [col for col in df_para_exportar_completo.columns 
                     if col not in columnas_ordenadas]

if columnas_restantes:
    print(f"Columnas adicionales ({len(columnas_restantes)}): {columnas_restantes}")
    columnas_ordenadas.extend(columnas_restantes)

print(f"\nTotal columnas para exportar: {len(columnas_ordenadas)}")

# Verificar que todas las columnas existen
columnas_finales_completas = [col for col in columnas_ordenadas 
                             if col in df_para_exportar_completo.columns]

print(f"Columnas confirmadas para exportar: {len(columnas_finales_completas)}")

# Preparar dataframe final
df_exportar_completo = df_para_exportar_completo[columnas_finales_completas].copy()

# Mostrar muestra de datos para verificar
print(f"\n=== MUESTRA DE DATOS FINALES ===")
print("Primeros 3 registros del dataframe a exportar:")
display(df_exportar_completo.head(3))

=== VERIFICAR COLUMNAS DISPONIBLES ===
Columnas disponibles en df_filtrado_puestos:
 1. fecha
 2. rut
 3. nombre
 4. apellidoPaterno
 5. apellidoMaterno
 6. PuestoIntegracion_trabajador
 7. observacion
 8. horaInicio
 9. horaProgramadaInicio
10. HoraInicioMarcaReloj
11. horaTermino
12. horaProgramadaTermino
13. HoraTerminoMarcaReloj
14. Centrocosto_trabajador
15. nombreMotivoPago
16. nombreTipoAusencia
17. Licencia_1ra_qcna
18. Licencias_periodo_1
19. Licencias_periodo_2
20. Permisos_periodo_1
21. Permisos_periodo_2
22. Ausencias_injust_periodo_1
23. Ausencias_injust_periodo_2
24. Atrasos_periodo_1
25. Atrasos_periodo_2
26. Omisiones_periodo_1
27. Omisiones_periodo_2
28. Olvido_entrada_p1
29. Olvido_salida_p1
30. Olvido_entrada_p2
31. Olvido_salida_p2
32. Olvidos_marcaje_periodo_1
33. Olvidos_marcaje_periodo_2

Total de columnas: 33
Total de registros: 5835
RUTs únicos: 283

Registros después de eliminar duplicados: 283

=== ORGANIZACIÓN DE COLUMNAS ===
Identificación (3): ['rut', 'nom

,rut,nombre,PuestoIntegracion_trabajador,Licencia_1ra_qcna,Licencias_periodo_1,Licencias_periodo_2,Permisos_periodo_1,Permisos_periodo_2,nombreTipoAusencia,Ausencias_injust_periodo_1,...,apellidoMaterno,observacion,horaInicio,horaProgramadaInicio,HoraInicioMarcaReloj,horaTermino,horaProgramadaTermino,HoraTerminoMarcaReloj,Centrocosto_trabajador,nombreMotivoPago
0,128294988,Cesar,Asistente De Servicios Generales,0,0,0,0,0,,0,...,Ibarra,,07:50:00,2025-07-07 07:50:00,2025-07-07 07:10:33,17:00:00,2025-07-07 17:00:00,2025-07-07 16:58:22,Administración,
21,256248786,Marie,Asistente De Servicios Generales,0,0,0,0,0,,0,...,,,11:00:00,2025-07-07 11:00:00,2025-07-07 10:48:43,20:30:00,2025-07-07 20:30:00,2025-07-07 20:32:25,Lavandería Producción,
26,107806466,Rosa,Asistente De Servicios Generales,0,0,0,0,0,,0,...,Carvallo,,06:50:00,2025-07-07 06:50:00,2025-07-07 06:19:53,15:30:00,2025-07-07 15:30:00,2025-07-07 15:32:43,Lavandería Producción,


In [44]:
## -- ESTE ES EL BUENO## -- 
# # EXPORTAR EXCEL COMPLETO CON TODAS LAS COLUMNAS 
print("=== EXPORTANDO EXCEL COMPLETO ===")

# Generar nombre de archivo con timestamp
timestamp = datetime.now().strftime("%d%m%Y_%H%M%S")
nombre_archivo_completo = f"Reporte_Bono_Asistencia_COMPLETO_{timestamp}.xlsx"
ruta_archivo_completo = f"c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/{nombre_archivo_completo}"

# Exportar a Excel con todas las columnas
df_exportar_completo.to_excel(ruta_archivo_completo, index=False)

print(f"✅ Archivo Excel COMPLETO generado exitosamente:")
print(f"📁 {ruta_archivo_completo}")
print(f"📊 Registros exportados: {len(df_exportar_completo)}")
print(f"📋 Columnas exportadas: {len(df_exportar_completo.columns)}")

# Verificar que el RUT problemático esté correcto
print(f"\n🔍 VERIFICACIÓN RUT 141944037 EN ARCHIVO COMPLETO:")
rut_verificacion_completo = df_exportar_completo[df_exportar_completo['rut'] == 141944037]

if len(rut_verificacion_completo) > 0:
    # Mostrar datos clave del RUT problema
    columnas_clave = ['rut', 'nombre', 'PuestoIntegracion_trabajador', 
                     'Licencias_periodo_1', 'Licencias_periodo_2',
                     'Permisos_periodo_1', 'Permisos_periodo_2',
                     'Ausencias_injust_periodo_1', 'Ausencias_injust_periodo_2',
                     'Atrasos_periodo_1', 'Atrasos_periodo_2',
                     'Olvido_entrada_p1', 'Olvido_salida_p1', 
                     'Olvido_entrada_p2', 'Olvido_salida_p2',
                     'Olvidos_marcaje_periodo_1', 'Olvidos_marcaje_periodo_2']
    
    print("Datos completos para RUT 141944037:")
    display(rut_verificacion_completo[columnas_clave])
    
    print(f"✅ CONFIRMADO: RUT 141944037 con olvidos correctos en el archivo completo")
else:
    print("❌ ERROR: RUT 141944037 no encontrado en exportación completa")

# Mostrar resumen estadístico del reporte completo
print(f"\n📈 RESUMEN ESTADÍSTICO COMPLETO:")
print(f"Total trabajadores: {len(df_exportar_completo)}")

# Estadísticas por período
for periodo in [1, 2]:
    print(f"\n--- PERÍODO {periodo} ---")
    col_licencias = f'Licencias_periodo_{periodo}'
    col_permisos = f'Permisos_periodo_{periodo}'
    col_ausencias = f'Ausencias_injust_periodo_{periodo}'
    col_atrasos = f'Atrasos_periodo_{periodo}'
    col_olvidos = f'Olvidos_marcaje_periodo_{periodo}'
    
    if col_licencias in df_exportar_completo.columns:
        print(f"  Licencias: {(df_exportar_completo[col_licencias] > 0).sum()} trabajadores con licencias")
    if col_permisos in df_exportar_completo.columns:
        print(f"  Permisos: {(df_exportar_completo[col_permisos] > 0).sum()} trabajadores con permisos")
    if col_ausencias in df_exportar_completo.columns:
        print(f"  Ausencias injustificadas: {(df_exportar_completo[col_ausencias] > 0).sum()} trabajadores")
    if col_atrasos in df_exportar_completo.columns:
        print(f"  Atrasos: {(df_exportar_completo[col_atrasos] > 0).sum()} trabajadores con atrasos")
    if col_olvidos in df_exportar_completo.columns:
        print(f"  Olvidos marcaje: {(df_exportar_completo[col_olvidos] > 0).sum()} trabajadores")

print(f"\n🎯 ARCHIVO EXCEL COMPLETO LISTO PARA ANÁLISIS DE BONO DE ASISTENCIA")
print(f"📋 Incluye TODAS las incidencias calculadas: licencias, permisos, ausencias, atrasos y olvidos")
print(f"✅ Datos corregidos y verificados para RUT 141944037")

# Mostrar estructura final del archivo
print(f"\n📁 ESTRUCTURA DEL ARCHIVO:")
print(f"  • Columnas de identificación: {len(columnas_identificacion)}")
print(f"  • Columnas de licencias: {len(columnas_licencias)}")
print(f"  • Columnas de permisos: {len(columnas_permisos)}")
print(f"  • Columnas de ausencias: {len(columnas_ausencias)}")
print(f"  • Columnas de tardanzas: {len(columnas_tardanzas)}")
print(f"  • Columnas de olvidos: {len(columnas_olvidos)}")
print(f"  • Columnas adicionales: {len(columnas_restantes)}")
print(f"  📊 TOTAL: {len(columnas_finales_completas)} columnas")

=== EXPORTANDO EXCEL COMPLETO ===

✅ Archivo Excel COMPLETO generado exitosamente:
📁 c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/Reporte_Bono_Asistencia_COMPLETO_13082025_101631.xlsx
📊 Registros exportados: 283
📋 Columnas exportadas: 33

🔍 VERIFICACIÓN RUT 141944037 EN ARCHIVO COMPLETO:
Datos completos para RUT 141944037:
✅ Archivo Excel COMPLETO generado exitosamente:
📁 c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/Reporte_Bono_Asistencia_COMPLETO_13082025_101631.xlsx
📊 Registros exportados: 283
📋 Columnas exportadas: 33

🔍 VERIFICACIÓN RUT 141944037 EN ARCHIVO COMPLETO:
Datos completos para RUT 141944037:


,rut,nombre,PuestoIntegracion_trabajador,Licencias_periodo_1,Licencias_periodo_2,Permisos_periodo_1,Permisos_periodo_2,Ausencias_injust_periodo_1,Ausencias_injust_periodo_2,Atrasos_periodo_1,Atrasos_periodo_2,Olvido_entrada_p1,Olvido_salida_p1,Olvido_entrada_p2,Olvido_salida_p2,Olvidos_marcaje_periodo_1,Olvidos_marcaje_periodo_2
1982,141944037,Manuel,Operario,0,0,0,0,0,0,2,2,1,1,2,0,2,2


✅ CONFIRMADO: RUT 141944037 con olvidos correctos en el archivo completo

📈 RESUMEN ESTADÍSTICO COMPLETO:
Total trabajadores: 283

--- PERÍODO 1 ---
  Licencias: 11 trabajadores con licencias
  Permisos: 3 trabajadores con permisos
  Ausencias injustificadas: 0 trabajadores
  Atrasos: 59 trabajadores con atrasos
  Olvidos marcaje: 24 trabajadores

--- PERÍODO 2 ---
  Licencias: 11 trabajadores con licencias
  Permisos: 5 trabajadores con permisos
  Ausencias injustificadas: 6 trabajadores
  Atrasos: 66 trabajadores con atrasos
  Olvidos marcaje: 32 trabajadores

🎯 ARCHIVO EXCEL COMPLETO LISTO PARA ANÁLISIS DE BONO DE ASISTENCIA
📋 Incluye TODAS las incidencias calculadas: licencias, permisos, ausencias, atrasos y olvidos
✅ Datos corregidos y verificados para RUT 141944037

📁 ESTRUCTURA DEL ARCHIVO:
  • Columnas de identificación: 3
  • Columnas de licencias: 3
  • Columnas de permisos: 2
  • Columnas de ausencias: 3
  • Columnas de tardanzas: 2
  • Columnas de olvidos: 8
  • Columnas ad

In [45]:
# GENERAR REPORTE DETALLADO DE INCIDENCIAS
print("=== GENERANDO REPORTE DETALLADO DE INCIDENCIAS ===")

# Crear DataFrame detallado con todas las incidencias encontradas
incidencias_detalladas = []

# Definir los tipos de incidencias que queremos capturar
tipos_licencia = ["Licencia", "Licencia Accidente Trayecto", "Licencia Accidente Trabajo", "Licencia Niño Menor", "Licencia postnatal"]
tipos_permisos = ["Permiso con goce de sueldo", "Permiso por Fallecimiento", "Permiso por matrimonio"]
ausencia_injustificada = ["Inasistencia"]

# Periodos definidos
periodo_1_inicio = pd.to_datetime('2025-07-06')
periodo_1_fin = pd.to_datetime('2025-07-20')
periodo_2_inicio = pd.to_datetime('2025-07-21')
periodo_2_fin = pd.to_datetime('2025-08-05')

print(f"Período 1: {periodo_1_inicio.strftime('%d-%m-%Y')} al {periodo_1_fin.strftime('%d-%m-%Y')}")
print(f"Período 2: {periodo_2_inicio.strftime('%d-%m-%Y')} al {periodo_2_fin.strftime('%d-%m-%Y')}")

# 1. PROCESAR LICENCIAS MÉDICAS
print("\n1. Procesando licencias médicas...")
licencias = df_filtrado_puestos[
    df_filtrado_puestos['nombreTipoAusencia'].isin(tipos_licencia)
].copy()

for _, row in licencias.iterrows():
    fecha = pd.to_datetime(row['fecha'])
    periodo = 1 if periodo_1_inicio <= fecha <= periodo_1_fin else (2 if periodo_2_inicio <= fecha <= periodo_2_fin else None)
    
    if periodo:
        incidencias_detalladas.append({
            'rut': row['rut'],
            'nombre_completo': f"{row['nombre']} {row['apellidoPaterno']} {row['apellidoMaterno']}",
            'puesto': row['PuestoIntegracion_trabajador'],
            'centro_costo': row['Centrocosto_trabajador'],
            'fecha': fecha.strftime('%d-%m-%Y'),
            'periodo': periodo,
            'tipo_incidencia': 'Licencia Médica',
            'subtipo': row['nombreTipoAusencia'],
            'detalle': f"Licencia médica: {row['nombreTipoAusencia']}",
            'observacion': row['observacion'] if pd.notna(row['observacion']) else '',
            'impacto_bono': 'Afecta cálculo de bono'
        })

print(f"   Licencias procesadas: {len([i for i in incidencias_detalladas if i['tipo_incidencia'] == 'Licencia Médica'])}")

# 2. PROCESAR PERMISOS
print("\n2. Procesando permisos...")
permisos = df_filtrado_puestos[
    df_filtrado_puestos['nombreTipoAusencia'].isin(tipos_permisos)
].copy()

for _, row in permisos.iterrows():
    fecha = pd.to_datetime(row['fecha'])
    periodo = 1 if periodo_1_inicio <= fecha <= periodo_1_fin else (2 if periodo_2_inicio <= fecha <= periodo_2_fin else None)
    
    if periodo:
        incidencias_detalladas.append({
            'rut': row['rut'],
            'nombre_completo': f"{row['nombre']} {row['apellidoPaterno']} {row['apellidoMaterno']}",
            'puesto': row['PuestoIntegracion_trabajador'],
            'centro_costo': row['Centrocosto_trabajador'],
            'fecha': fecha.strftime('%d-%m-%Y'),
            'periodo': periodo,
            'tipo_incidencia': 'Permiso',
            'subtipo': row['nombreTipoAusencia'],
            'detalle': f"Permiso: {row['nombreTipoAusencia']}",
            'observacion': row['observacion'] if pd.notna(row['observacion']) else '',
            'impacto_bono': 'Puede afectar cálculo según política'
        })

print(f"   Permisos procesados: {len([i for i in incidencias_detalladas if i['tipo_incidencia'] == 'Permiso'])}")

# 3. PROCESAR AUSENCIAS INJUSTIFICADAS
print("\n3. Procesando ausencias injustificadas...")
ausencias = df_filtrado_puestos[
    df_filtrado_puestos['nombreTipoAusencia'].isin(ausencia_injustificada)
].copy()

for _, row in ausencias.iterrows():
    fecha = pd.to_datetime(row['fecha'])
    periodo = 1 if periodo_1_inicio <= fecha <= periodo_1_fin else (2 if periodo_2_inicio <= fecha <= periodo_2_fin else None)
    
    if periodo:
        incidencias_detalladas.append({
            'rut': row['rut'],
            'nombre_completo': f"{row['nombre']} {row['apellidoPaterno']} {row['apellidoMaterno']}",
            'puesto': row['PuestoIntegracion_trabajador'],
            'centro_costo': row['Centrocosto_trabajador'],
            'fecha': fecha.strftime('%d-%m-%Y'),
            'periodo': periodo,
            'tipo_incidencia': 'Ausencia Injustificada',
            'subtipo': row['nombreTipoAusencia'],
            'detalle': f"Inasistencia sin justificación",
            'observacion': row['observacion'] if pd.notna(row['observacion']) else '',
            'impacto_bono': 'Afecta negativamente el bono'
        })

print(f"   Ausencias injustificadas procesadas: {len([i for i in incidencias_detalladas if i['tipo_incidencia'] == 'Ausencia Injustificada'])}")

# 4. PROCESAR ATRASOS (del dataframe df_atrasos si existe)
print("\n4. Procesando atrasos...")
if 'df_atrasos' in locals():
    atrasos = df_atrasos[df_atrasos['tipo_asistencia'] == 'Atraso'].copy()
    
    for _, row in atrasos.iterrows():
        fecha = pd.to_datetime(row['fecha'])
        periodo = 1 if periodo_1_inicio <= fecha <= periodo_1_fin else (2 if periodo_2_inicio <= fecha <= periodo_2_fin else None)
        
        if periodo:
            incidencias_detalladas.append({
                'rut': row['rut'],
                'nombre_completo': f"{row['nombre']} {row['apellidoPaterno']} {row['apellidoMaterno']}" if pd.notna(row['apellidoPaterno']) else row['nombre'],
                'puesto': row['PuestoIntegracion_trabajador'],
                'centro_costo': row['Centrocosto_trabajador'],
                'fecha': fecha.strftime('%d-%m-%Y'),
                'periodo': periodo,
                'tipo_incidencia': 'Atraso',
                'subtipo': f"Atraso de {row['atraso_minutos']:.0f} minutos",
                'detalle': f"Llegada tarde - {row['atraso_minutos']:.0f} minutos de atraso",
                'observacion': f"Hora programada: {row['horaProgramadaInicio'].strftime('%H:%M') if pd.notna(row['horaProgramadaInicio']) else 'N/A'}, Hora real: {row['HoraInicioMarcaReloj'].strftime('%H:%M') if pd.notna(row['HoraInicioMarcaReloj']) else 'N/A'}",
                'impacto_bono': 'Afecta cálculo según política de tardanzas'
            })

print(f"   Atrasos procesados: {len([i for i in incidencias_detalladas if i['tipo_incidencia'] == 'Atraso'])}")

# 5. PROCESAR OMISIONES DE MARCA (del dataframe df_atrasos si existe)
print("\n5. Procesando omisiones de marca...")
if 'df_atrasos' in locals():
    omisiones = df_atrasos[df_atrasos['tipo_asistencia'] == 'Omisión de marca'].copy()
    
    for _, row in omisiones.iterrows():
        fecha = pd.to_datetime(row['fecha'])
        periodo = 1 if periodo_1_inicio <= fecha <= periodo_1_fin else (2 if periodo_2_inicio <= fecha <= periodo_2_fin else None)
        
        if periodo:
            incidencias_detalladas.append({
                'rut': row['rut'],
                'nombre_completo': f"{row['nombre']} {row['apellidoPaterno']} {row['apellidoMaterno']}" if pd.notna(row['apellidoPaterno']) else row['nombre'],
                'puesto': row['PuestoIntegracion_trabajador'],
                'centro_costo': row['Centrocosto_trabajador'],
                'fecha': fecha.strftime('%d-%m-%Y'),
                'periodo': periodo,
                'tipo_incidencia': 'Omisión de Marca',
                'subtipo': 'Falta marcaje en reloj',
                'detalle': f"No registró marca en el reloj biométrico",
                'observacion': f"Hora programada: {row['horaProgramadaInicio'].strftime('%H:%M') if pd.notna(row['horaProgramadaInicio']) else 'N/A'}",
                'impacto_bono': 'Puede afectar cálculo según política'
            })

print(f"   Omisiones procesadas: {len([i for i in incidencias_detalladas if i['tipo_incidencia'] == 'Omisión de Marca'])}")

# 6. PROCESAR OLVIDOS DE MARCAJE (del reporte externo)
print("\n6. Procesando olvidos de marcaje del reporte externo...")
if 'df_reporte_omisiones' in locals() and 'Rut_limpio' in df_reporte_omisiones.columns:
    # Convertir RUT para matching
    df_reporte_omisiones['rut_match'] = df_reporte_omisiones['Rut_limpio'].astype(str)
    
    # Crear diccionario de trabajadores para lookup rápido
    trabajadores_dict = df_filtrado_puestos.drop_duplicates('rut').set_index('rut').to_dict('index')
    
    for _, row in df_reporte_omisiones.iterrows():
        if pd.notna(row['Rut_limpio']):
            fecha = pd.to_datetime(row[columna_fecha_omisiones])
            periodo = 1 if periodo_1_inicio <= fecha <= periodo_1_fin else (2 if periodo_2_inicio <= fecha <= periodo_2_fin else None)
            
            if periodo:
                rut = int(row['Rut_limpio'])
                trabajador = trabajadores_dict.get(rut, {})
                
                # Determinar tipo de olvido
                motivos = []
                if columna_entrada and pd.notna(row[columna_entrada]) and row[columna_entrada] == "Olvido de Marcaje":
                    motivos.append("Entrada")
                if columna_salida and pd.notna(row[columna_salida]) and row[columna_salida] == "Olvido de Marcaje":
                    motivos.append("Salida")
                
                for motivo in motivos:
                    incidencias_detalladas.append({
                        'rut': rut,
                        'nombre_completo': trabajador.get('nombre', 'N/A'),
                        'puesto': trabajador.get('PuestoIntegracion_trabajador', 'N/A'),
                        'centro_costo': trabajador.get('Centrocosto_trabajador', 'N/A'),
                        'fecha': fecha.strftime('%d-%m-%Y'),
                        'periodo': periodo,
                        'tipo_incidencia': 'Olvido de Marcaje',
                        'subtipo': f"Olvido marcaje {motivo.lower()}",
                        'detalle': f"Olvido de marcaje en {motivo.lower()} reportado en sistema",
                        'observacion': f"Reportado en sistema de marcajes",
                        'impacto_bono': 'Afecta cálculo según política'
                    })

print(f"   Olvidos de marcaje procesados: {len([i for i in incidencias_detalladas if i['tipo_incidencia'] == 'Olvido de Marcaje'])}")

# Crear DataFrame con todas las incidencias
df_incidencias = pd.DataFrame(incidencias_detalladas)

print(f"\n=== RESUMEN TOTAL DE INCIDENCIAS ===")
print(f"Total de incidencias registradas: {len(df_incidencias)}")

if len(df_incidencias) > 0:
    print(f"\nPor tipo de incidencia:")
    resumen_tipos = df_incidencias['tipo_incidencia'].value_counts()
    for tipo, cantidad in resumen_tipos.items():
        print(f"  {tipo}: {cantidad}")
    
    print(f"\nPor período:")
    resumen_periodos = df_incidencias['periodo'].value_counts().sort_index()
    for periodo, cantidad in resumen_periodos.items():
        print(f"  Período {periodo}: {cantidad}")
    
    print(f"\nTrabajadores únicos con incidencias: {df_incidencias['rut'].nunique()}")
    
    # Ordenar por fecha y trabajador
    df_incidencias = df_incidencias.sort_values(['fecha', 'nombre_completo', 'tipo_incidencia'])
    
    # Generar archivo Excel
    timestamp = datetime.now().strftime("%d%m%Y_%H%M%S")
    nombre_archivo_incidencias = f"Reporte_Detalle_Incidencias_Bono_Asistencia_{timestamp}.xlsx"
    ruta_archivo_incidencias = f"c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/{nombre_archivo_incidencias}"
    
    # Exportar con formato mejorado
    with pd.ExcelWriter(ruta_archivo_incidencias, engine='xlsxwriter') as writer:
        # Hoja principal con todas las incidencias
        df_incidencias.to_excel(writer, sheet_name='Detalle_Incidencias', index=False)
        
        # Hoja resumen por trabajador
        resumen_por_trabajador = df_incidencias.groupby(['rut', 'nombre_completo', 'puesto']).agg({
            'tipo_incidencia': 'count',
            'periodo': lambda x: list(x.unique())
        }).rename(columns={'tipo_incidencia': 'total_incidencias'}).reset_index()
        
        resumen_por_trabajador.to_excel(writer, sheet_name='Resumen_por_Trabajador', index=False)
        
        # Hoja resumen por tipo
        resumen_por_tipo = df_incidencias.groupby(['tipo_incidencia', 'periodo']).agg({
            'rut': 'count',
            'nombre_completo': 'nunique'
        }).rename(columns={'rut': 'total_incidencias', 'nombre_completo': 'trabajadores_afectados'}).reset_index()
        
        resumen_por_tipo.to_excel(writer, sheet_name='Resumen_por_Tipo', index=False)
    
    print(f"\n✅ REPORTE DE INCIDENCIAS GENERADO EXITOSAMENTE:")
    print(f"📁 {ruta_archivo_incidencias}")
    print(f"📊 Contiene {len(df_incidencias)} incidencias detalladas")
    print(f"📋 Incluye 3 hojas: Detalle, Resumen por Trabajador, Resumen por Tipo")
    
    # Mostrar muestra de datos
    print(f"\n=== MUESTRA DE INCIDENCIAS ===")
    display(df_incidencias.head(10))
    
else:
    print("❌ No se encontraron incidencias en los períodos analizados")

print(f"\n🎯 REPORTE ESPECÍFICO DE INCIDENCIAS COMPLETADO")
print(f"📈 Este archivo complementa el reporte de bono principal")
print(f"🔍 Contiene el detalle granular de cada incidencia encontrada")

=== GENERANDO REPORTE DETALLADO DE INCIDENCIAS ===
Período 1: 06-07-2025 al 20-07-2025
Período 2: 21-07-2025 al 05-08-2025

1. Procesando licencias médicas...
   Licencias procesadas: 189

2. Procesando permisos...
   Permisos procesados: 13

3. Procesando ausencias injustificadas...
   Ausencias injustificadas procesadas: 6

4. Procesando atrasos...
   Atrasos procesados: 260

5. Procesando omisiones de marca...
   Omisiones procesadas: 3

6. Procesando olvidos de marcaje del reporte externo...
   Olvidos de marcaje procesados: 525

=== RESUMEN TOTAL DE INCIDENCIAS ===
Total de incidencias registradas: 996

Por tipo de incidencia:
  Olvido de Marcaje: 525
  Atraso: 260
  Licencia Médica: 189
  Permiso: 13
  Ausencia Injustificada: 6
  Omisión de Marca: 3

Por período:
  Período 1: 407
  Período 2: 589

Trabajadores únicos con incidencias: 268

✅ REPORTE DE INCIDENCIAS GENERADO EXITOSAMENTE:
📁 c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/Reporte_Detalle_Incidencias_Bo

,rut,nombre_completo,puesto,centro_costo,fecha,periodo,tipo_incidencia,subtipo,detalle,observacion,impacto_bono
965,207617652,Alejandro,Operario Almacenamiento y Gestión de Residuos,Prev. De Riesgo Y Medio Ambiente,01-08-2025,2,Olvido de Marcaje,Olvido marcaje salida,Olvido de marcaje en salida reportado en sistema,Reportado en sistema de marcajes,Afecta cálculo según política
371,207617652,Alejandro Ayala Salas,Operario Almacenamiento y Gestión de Residuos,Prev. De Riesgo Y Medio Ambiente,01-08-2025,2,Atraso,Atraso de 3 minutos,Llegada tarde - 3 minutos de atraso,"Hora programada: 07:50, Hora real: 08:02",Afecta cálculo según política de tardanzas
385,159344169,Alexander Araos Deramond,Operario,Fabricación Y Envasado Mezclas Polvos,01-08-2025,2,Atraso,Atraso de 12 minutos,Llegada tarde - 12 minutos de atraso,"Hora programada: 05:50, Hora real: 06:11",Afecta cálculo según política de tardanzas
478,77734899,Angel,Operario,Envasado Bodega Sabores,01-08-2025,2,Olvido de Marcaje,Olvido marcaje entrada,Olvido de marcaje en entrada reportado en sistema,Reportado en sistema de marcajes,Afecta cálculo según política
986,201913616,Benjamin,Asistente De Laboratorio,Desarrollo Sabores Salados,01-08-2025,2,Olvido de Marcaje,Olvido marcaje entrada,Olvido de marcaje en entrada reportado en sistema,Reportado en sistema de marcajes,Afecta cálculo según política
155,99010339,Claudio Gutierrez Caceres,Operario,Planta Secado 2,01-08-2025,2,Licencia Médica,Licencia,Licencia médica: Licencia,,Afecta cálculo de bono
463,204332444,Consuelo Orellana Díaz,Asistente De Laboratorio,Desarrollo Sabores Dulces,01-08-2025,2,Atraso,Atraso de 10 minutos,Llegada tarde - 10 minutos de atraso,"Hora programada: 07:50, Hora real: 08:10",Afecta cálculo según política de tardanzas
306,164097072,Crisch Rebolledo Cariz,Asistente De Laboratorio,Desarrollo Sabores General,01-08-2025,2,Atraso,Atraso de 5 minutos,Llegada tarde - 5 minutos de atraso,"Hora programada: 07:50, Hora real: 08:04",Afecta cálculo según política de tardanzas
393,201626455,Daniel Adonis Adones,Asistente De Laboratorio,Muestras Sabores,01-08-2025,2,Atraso,Atraso de 6 minutos,Llegada tarde - 6 minutos de atraso,"Hora programada: 07:50, Hora real: 08:06",Afecta cálculo según política de tardanzas
342,274066415,Daniel Torrealba Mujica,Operario,Envasado Bodega Sabores,01-08-2025,2,Atraso,Atraso de 2 minutos,Llegada tarde - 2 minutos de atraso,"Hora programada: 07:50, Hora real: 08:02",Afecta cálculo según política de tardanzas



🎯 REPORTE ESPECÍFICO DE INCIDENCIAS COMPLETADO
📈 Este archivo complementa el reporte de bono principal
🔍 Contiene el detalle granular de cada incidencia encontrada


In [46]:
# VERSIÓN ALTERNATIVA SIN XLSXWRITER (por si hay problemas)
print("=== VERSIÓN ALTERNATIVA: REPORTE DE INCIDENCIAS SIN XLSXWRITER ===")

# Si no tienes xlsxwriter instalado, puedes usar esta versión que usa openpyxl (que ya está instalado)
def generar_reporte_incidencias_alternativo():
    """
    Genera el reporte de incidencias usando openpyxl en lugar de xlsxwriter
    """
    
    # Reutilizar el mismo DataFrame de incidencias si ya existe
    if 'df_incidencias' in locals() and len(df_incidencias) > 0:
        print(f"Usando DataFrame existente con {len(df_incidencias)} incidencias")
        df_para_exportar = df_incidencias.copy()
    else:
        print("Creando nuevo DataFrame de incidencias...")
        # Aquí iría el mismo código de procesamiento que antes
        # Por simplicidad, asumo que df_incidencias ya existe
        print("❌ No hay DataFrame de incidencias disponible. Ejecuta primero la celda anterior.")
        return
    
    # Generar archivo Excel simple
    timestamp = datetime.now().strftime("%d%m%Y_%H%M%S")
    nombre_archivo_simple = f"Reporte_Incidencias_Simple_{timestamp}.xlsx"
    ruta_archivo_simple = f"c:/Users/bgacitua/Desktop/Scripts de Python/Reportes generados/{nombre_archivo_simple}"
    
    # Exportar usando el motor por defecto (openpyxl)
    try:
        # Archivo principal
        df_para_exportar.to_excel(ruta_archivo_simple, index=False, sheet_name='Incidencias')
        
        print(f"✅ REPORTE ALTERNATIVO GENERADO:")
        print(f"📁 {ruta_archivo_simple}")
        print(f"📊 {len(df_para_exportar)} incidencias exportadas")
        
        # Crear también archivos CSV como backup
        csv_filename = ruta_archivo_simple.replace('.xlsx', '.csv')
        df_para_exportar.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        print(f"📋 También se generó archivo CSV: {csv_filename}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error al generar archivo: {e}")
        print("💡 Intenta ejecutar: pip install openpyxl")
        return False

# Ejecutar la función alternativa
resultado = generar_reporte_incidencias_alternativo()

if resultado:
    print(f"\n🎯 REPORTE GENERADO EXITOSAMENTE")
    print(f"💡 Si necesitas múltiples hojas en Excel, asegúrate de tener xlsxwriter instalado")
else:
    print(f"\n❌ No se pudo generar el reporte alternativo")
    print(f"💡 Ejecuta la celda anterior para crear el DataFrame de incidencias primero")

=== VERSIÓN ALTERNATIVA: REPORTE DE INCIDENCIAS SIN XLSXWRITER ===
Creando nuevo DataFrame de incidencias...
❌ No hay DataFrame de incidencias disponible. Ejecuta primero la celda anterior.

❌ No se pudo generar el reporte alternativo
💡 Ejecuta la celda anterior para crear el DataFrame de incidencias primero
